# <center>大模型 AI Agent 开发实战

## <center>Ch.16 MicroSoft AutoGen 代理对话与人机交互源码解析

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501071618677.png" width=100%></div>

&emsp;&emsp;在《Ch.15 MicroSoft AutoGen 开发框架基础入门》第一节课中，我们主要围绕框架中的核心类——`ConversableAgent`进行了讲解。作为对话式代理的基类（内置组件），深入理解其内部执行逻辑对于掌握`AutoGen`框架至关重要，能够显著提升我们对该框架的理解深度和使用熟练度。特别是`ConversableAgent`类中的`__init__`方法，其每个参数都代表了对话式交互中的关键功能，如下所示：

```python
    def __init__(
        self,
        name: str,
        system_message: Optional[Union[str, List]] = "You are a helpful AI Assistant.",
        is_termination_msg: Optional[Callable[[Dict], bool]] = None,
        max_consecutive_auto_reply: Optional[int] = None,
        human_input_mode: Literal["ALWAYS", "NEVER", "TERMINATE"] = "TERMINATE",
        function_map: Optional[Dict[str, Callable]] = None,
        code_execution_config: Union[Dict, Literal[False]] = False,
        llm_config: Optional[Union[Dict, Literal[False]]] = None,
        default_auto_reply: Union[str, Dict] = "",
        description: Optional[str] = None,
        chat_messages: Optional[Dict[Agent, List[Dict]]] = None,
    ):
```

&emsp;&emsp;回顾上节课介绍的知识点，我们重点介绍了`name`、`system_message`、`description`的意义，如何通过`llm_config`接入不同的大模型实例（在线大模型和开源大模型），以及使用`code_execution_config`配置不同形式的代码执行器，并且实际通过代码实操了接入本地代码执行和`Docker`容器内执行代码的两种方法。在`AutoGen`中与大模型进行对话交互，也通过源码的解读介绍了`ConversableAgent`类中的`generate_reply`方法内部生成回复的执行逻辑。

&emsp;&emsp;而在第一节课最后的案例中，我们构建了一个自动Debug的数据分析器，让两个代理通过对话的方式来共同解决数据分析问题。而在这个过程中，我们使用了一个全新的方法`initiate_chat`方法来执行过程，那么该方法内部的机制和执行逻辑是怎么样的呢？ 我们就在本节课给大家展开详细的介绍。

&emsp;&emsp;同时，对于`ConversableAgent`类`__init__`方法中遗留的参数：`is_termination_msg`、`max_consecutive_auto_reply`、`human_input_mode`、`function_map`、`default_auto_reply`和`chat_messages`逐一展开详细的讲解，这几个参数直接对应着构建对话式`Multi-Agent`架构模式，以及人工交互、函数调用等重要的交互逻辑，

&emsp;&emsp;首先，我们来看`initiate_chat`函数的内部执行逻辑。

# 1. initiate_chat源码解读

&emsp;&emsp;`ConversableAgent`中的 **`initiate_chat` 方法是 `AutoGen` 框架中非常关键且核心的方法，用于初始化和启动多智能体聊天流程，包括消息传递、任务分配以及对历史对话的处理。** 我们就以双智能体对话为例分析它的流程，如下图所示：

> 图片来源于 AutoGen 官方文档：https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501071058343.png" width=100%></div>

&emsp;&emsp;在上述流程图中，可以看到以下流程：
1. **输入消息与上下文**：用户发送一个问题或消息（如“什么是三角不等式？”），初始化器（`Initializer`）接收消息，并可能结合上下文对消息进行预处理；
2. **智能体交互**：`Agent A` 和 `Agent B` 接收来自初始化器的消息，并进行多轮的双向聊天（`Chat`），通过推理、协作等方式完善答案；
3. **历史记录与总结**：对于聊天的历史记录，经过智能体交互后被传递到一个总结器（`Summarizer`），总结器使用一些内置的方法进行摘要总结；
4. **输出结果**：最后，`Summarizer` 输出一个清晰、凝练的结果作为对用户问题的回应。

&emsp;&emsp;其源码位置如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501071703219.png" width=100%></div>

```python
    def initiate_chat(
        self,
        recipient: "ConversableAgent",
        clear_history: bool = True,
        silent: Optional[bool] = False,
        cache: Optional[AbstractCache] = None,
        max_turns: Optional[int] = None,
        summary_method: Optional[Union[str, Callable]] = DEFAULT_SUMMARY_METHOD,
        summary_args: Optional[dict] = {},
        message: Optional[Union[Dict, str, Callable]] = None,
        **kwargs,
    ) -> ChatResult:
```

&emsp;&emsp;`initiate_chat`方法也包含一些初始化参数，这些参数用于设置对话模式并控制聊天的行为。我们可以逐步分析这个方法的功能和作用。

| 参数名称              | 类型                            | 说明                                                                                 | 默认值              |
|-------------------|-------------------------------|------------------------------------------------------------------------------------|-------------------|
| `recipient`       | `ConversableAgent`             | 与之发起对话的目标智能体。                                                              | 无默认值           |
| `clear_history`   | `bool`                         | 是否清除与接收方的历史对话记录。默认为 `True`，即清除历史。                           | `True`            |
| `silent`          | `Optional[bool]`               | 控制是否打印对话消息（用于调试）。默认为 `False`，表示打印消息。                       | `False`           |
| `cache`           | `Optional[AbstractCache]`      | 指定用于对话的缓存客户端（如内存、数据库缓存）。默认为 `None`。                       | `None`            |
| `max_turns`       | `Optional[int]`                | 对话的最大回合数。回合数指的是两方交互的次数。如果为 `None`，则表示没有回合数限制。    | `None`            |
| `summary_method`  | `Optional[Union[str, Callable]]` | 用于从对话中提取总结的方法。支持字符串 `"last_msg"` 或 `"reflection_with_llm"`，也可以是自定义的 callable 方法。 | `DEFAULT_SUMMARY_METHOD` (`"last_msg"`) |
| `summary_args`    | `Optional[dict]`               | 传递给 `summary_method` 的额外参数，例如 `summary_prompt` 和 `summary_role` 等。        | `{}`              |
| `message`         | `Optional[Union[Dict, str, Callable]]` | 初始消息，传递给接收方的第一条消息。可以是字符串、字典或 callable。字典可以包含 `content` 或 `tool_calls`。 | 无默认值           |

&emsp;&emsp;我们通过一个对话示例来理解各个参数的作用及意义。这里我们接入DeepSeek v3 模型测试。

- DeepSeek v3账号注册与API获取

&emsp;&emsp;`DeepSeek`官网：https://www.deepseek.com/

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250107155848673.png" alt="image-20250107155848673" style="zoom:33%;" />

&emsp;&emsp;新用户注册即赠送10元额度，约500万token额度（如果发现没有赠送额度，那就是官方停止了活动。）

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250107155925049.png" alt="image-20250107155925049" style="zoom:25%;" />

&emsp;&emsp;对比`GPT4o`价格，约降低`90%`以上：输入价格为`GPT4o`的`6%`，输出价格为`GPT4o`的`3%`。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250107160101536.png" alt="image-20250107160101536" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250107160418665.png" alt="image-20250107160418665" style="zoom: 50%;" />

&emsp;&emsp;而且其`API`调用不限速：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250107160659366.png" alt="image-20250107160659366" style="zoom:33%;" />

&emsp;&emsp;最关键的是，调用风格和`OpenAI`完全一致：`Function calling`、`Json Output`等功能完全相同：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250107161014344.png" alt="image-20250107161014344" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250107161058708.png" alt="image-20250107161058708" style="zoom:33%;" />

In [1]:
import os

from autogen import ConversableAgent

&emsp;&emsp;定义可接入大模型的配置列表：

In [3]:
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            "api_key": os.environ.get("OPENAI_API_KEY"),
            "tags": ["openai"]
        },
        {
            "model": "qwen2.5:32b",
            "base_url": "http://192.168.110.131:11434/v1/",   # 注意：如果使用Ollama模型，请先启动模型服务，在替换为自己的实际访问地址
            "price": [0.00, 0.00],
            "tags": ["ollama"]
        },
        {
            "model": "qwen2.5-coder:32b",
            "base_url": "http://192.168.110.131:11434/v1/",  # 注意：如果使用Ollama模型，请先启动模型服务，在替换为自己的实际访问地址
            "price": [0.00, 0.00],
            "tags": ["ollama-coder"]
        },
        {
            "model": "deepseek-chat",
            "api_key":"sk-7328ed07b5c647eb93c935e1c361c0da",   # 这里替换成自己的key
            "base_url": "https://api.deepseek.com",
            "price": [0.00, 0.00],
            "tags": ["deepseek"]
        }
    ]
}

&emsp;&emsp;通过`tags`过滤选择的模型，这里我们使用`deepseek` 的 `deepseek-chat`模型作为代理模型。

In [4]:
import autogen

# 过滤出包含 'ollama' 标签的模型配置
filter_model = {"tags": ["deepseek"]}

config_model = autogen.filter_config(
    config_list=llm_config["config_list"], 
    filter_dict=filter_model)

&emsp;&emsp;我们希望通过两个智能体（Agent 1 和 Agent 2）的对话，协作回答一个问题。因此定义出两个`ConversableAgent`实例。

In [5]:
# 定义了一个agent 1
agent1 = ConversableAgent(
    "agent1",
    llm_config={
        "cache_seed": None,  # 禁用缓存
        "config_list": config_model},  # 这里使用 config_model
)

agent2 = ConversableAgent(
    "agent2",
    llm_config={
        "cache_seed": None,  # 禁用缓存
        "config_list": config_model},  # 这里使用 config_model
)

## 1.1 message 参数解析

&emsp;&emsp;`initiate_chat`参数必须指定的两个参数是`recipient`和`message`，前者表示消息的接收者，也就是要与谁对话，后者表示对话的初始消息，即多轮对话提出的问题。同时，**需要注意的是：在这种双智能体或多智能体对话场景中，必须考虑对话的终止条件，否则会导致对话陷入无休止的循环。** 最简单的一个参数是`max_turns`，通过限制对话的最大轮数（turns）来控制对话的长度，每一轮对话包括一个发起者和一个响应者的完整交互。因此代码如下图所示：

In [6]:
# 启动一个对话
response = agent1.initiate_chat(
    recipient=agent2,
    message="什么是黑洞？你能用通俗的语言解释吗？",
    max_turns=2
)

print(response)  # 输出对话的响应

agent1 (to agent2):

什么是黑洞？你能用通俗的语言解释吗？

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1):

当然可以！黑洞是宇宙中一种非常神秘的天体，它的引力极其强大，强大到连光都无法逃脱。你可以把它想象成一个“宇宙漩涡”，任何靠近它的东西，包括光，都会被吸进去，再也出不来。

黑洞的形成通常是因为一颗非常大的恒星在生命结束时发生了剧烈的坍缩。恒星的核心在自身引力的作用下压缩成一个极其密集的点，这个点的密度和引力都变得无限大，形成了黑洞。

黑洞的边界叫做“事件视界”，你可以把它理解为一个“不归点”。一旦任何东西跨过这个边界，就再也无法逃脱了。

虽然黑洞本身是看不见的（因为它连光都能吸收），但科学家可以通过观察它周围物质的运动来推断它的存在。比如，当物质被吸入黑洞时，会形成一个旋转的盘状结构，这个盘会发出强烈的辐射，从而让我们间接“看到”黑洞。

总之，黑洞是宇宙中最极端的天体之一，它的存在挑战了我们对物理学的理解，也激发了无数科学家和科幻爱好者的想象力！

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent1 (to agent2):

你解释得非常清楚！黑洞确实是宇宙中最令人着迷的天体之一。让我再补充一些有趣的点，帮助你更全面地理解黑洞：

1. **黑洞的大小**：黑洞的大小可以相差很大。有些黑洞的质量只有太阳的几倍，而有些超大质量黑洞的质量可以达到太阳的数十亿倍！这些超大质量黑洞通常位于星系的中心，比如我们银河系的中心就有一个超大质量黑洞。

2. **时间扭曲**：由于黑洞的引力极其强大，它还会扭曲时间和空间。如果你能靠近黑洞（当然，这是非常危险的！），你会感觉到时间变慢了。这种现象叫做“时间膨胀”，是爱因斯坦广义相对论的一个重要预测。

3. **霍金辐射**：虽然黑洞被认为是“只进不出”的，但著名物理学家斯蒂芬·霍金提出，黑洞其实会通过一

&emsp;&emsp;其中，要发送给接收方的初始消息`message`必须提供，如果不提供，`initiate_chat`方法中定义的逻辑会调用 `input()` 来获取输入，比如：

In [8]:
# 启动一个对话
response = agent1.initiate_chat(
    recipient=agent2,
    max_turns=2
)

print(response)  # 输出对话的响应

> 什么是 黑洞？


agent1 (to agent2):

什么是 黑洞？

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1):

黑洞是宇宙中一种极其密集的天体，其引力强大到连光都无法逃脱。黑洞的形成通常与恒星的生命周期有关。当一颗质量非常大的恒星耗尽了核燃料后，会发生引力坍缩，核心部分压缩成一个极其致密的点，称为奇点。这个奇点的引力场非常强大，以至于在其周围形成一个称为“事件视界”的边界。一旦物质或光越过这个边界，就无法逃脱，因此我们无法直接“看到”黑洞，只能通过观察其周围物质的运动和辐射来推断其存在。

黑洞的主要特征包括：

1. **奇点**：黑洞的中心是一个密度无限大、体积无限小的点，称为奇点。在这里，已知的物理定律可能不再适用。

2. **事件视界**：这是黑洞的边界，任何进入事件视界的物质或光都无法逃脱。事件视界的半径称为“史瓦西半径”。

3. **引力场**：黑洞的引力场极其强大，能够扭曲周围的时空，甚至影响光线的传播路径。

4. **吸积盘**：当黑洞吸引周围的物质时，这些物质会在黑洞周围形成一个旋转的盘状结构，称为吸积盘。吸积盘中的物质在落入黑洞之前会因摩擦和引力作用而发热，发出强烈的辐射，通常包括X射线。

5. **喷流**：一些黑洞，特别是超大质量黑洞，会从其两极发射出高速的等离子体喷流，这些喷流可以延伸到数千光年之外。

黑洞根据质量可以分为几类：

- **恒星质量黑洞**：质量大约是太阳的几倍到几十倍，通常由大质量恒星坍缩形成。
- **中等质量黑洞**：质量介于恒星质量黑洞和超大质量黑洞之间，目前观测到的证据较少。
- **超大质量黑洞**：质量可以达到太阳的百万倍甚至数十亿倍，通常位于星系的中心。例如，银河系中心就有一个质量约为太阳400万倍的超大质量黑洞。

黑洞的研究对于理解宇宙的演化、引力理论以及极端条件下的物理现象具有重要意义。

--------------------------------------------------------------------------------

>>>>>>>> USING AU

&emsp;&emsp;除此以外，`message`如果是字典或者字符串，则可以直接作为初始消息使用，如果是字典，可以包含：
- content: 消息内容，可以为 None。
- tool_calls": 包含函数名称和参数的字典列表。
- role": 消息的角色，可以是 "assistant", "user", 或 "function"。
- name": 通常不需要，除非角色是 "function"，用于指示函数名称。
- context": 消息的上下文，将传递给 OpenAIWrapper.create。

&emsp;&emsp;这里我们关注 `context` 参数，该参数中的内容用于传递额外的上下文信息给消息生成和摘要方法。这些信息可以在生成初始消息或生成对话摘要时使用。源码逻辑如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501071828643.png" width=100%></div>

&emsp;&emsp;这里`_`代表当前的对话回合数（turn number）。`_ == 0 `表示这是对话的第一个回合，即初始化消息，它会检查 `message` 参数是否是一个可调用对象（Callable），即函数或实现了 `__call__` 方法的对象。注意：在 `Python` 中，`Callable` 指的是可以调用的对象，通常是函数或实现了 `__call__` 方法的类实例。它允许你在运行时动态生成消息内容，而不仅仅依赖于静态的字符串或字典。使用 `Callable` 作为 `message` 参数可以提供更大的灵活性和动态性。例如，你可以根据不同的上下文、用户信息或其他条件生成不同的初始消息，而不需要预先定义所有可能的消息内容。

&emsp;&emsp;要自定义 `Callable`，我们需要定义一个函数（或实现 `__call__` 方法的类），该函数接受发送方代理、接收方代理和上下文参数，并返回一个字符串或字典作为初始消息，这里我们可以给出一个示例代码用来自定义初始消息函数，并返回字典。代码如下：

In [9]:
from typing import Union, Dict

# 定义自定义的初始消息函数
def my_custom_message_dict(sender: ConversableAgent, recipient: ConversableAgent, context: dict) -> Union[str, Dict]:
    # 从上下文中获取用户的兴趣
    user_interest = context.get("user_interest", "技术")
    
    # 构建定制化的消息内容
    message = {
        'role': 'user',
        'content': f"你好！我听说你对 {user_interest} 很感兴趣。我们开始聊天吧！",
        # 如果需要，可以将额外信息包含在上下文中
        # 'context': additional_info  # 通常不需要在消息中包含 'context' 字段
    }
    
    return message

&emsp;&emsp;然后使用自定义 `Callable` 启动聊天会话。

In [10]:
# 定义上下文信息
context_info = {
    "user_interest": "机器学习",
    "additional_info": "这是一个测试对话。"
}

# 启动聊天会话，使用自定义初始消息函数
response = agent1.initiate_chat(
    recipient=agent2,
    message=my_custom_message_dict,  # 传入自定义函数
    max_turns=2,
    **context_info,
)

print(response)

agent1 (to agent2):

你好！我听说你对 机器学习 很感兴趣。我们开始聊天吧！

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1):

你好！很高兴和你聊天！是的，我对机器学习非常感兴趣。机器学习是一个非常广泛的领域，涵盖了从基础算法到实际应用的各个方面。你对机器学习的哪个方面感兴趣呢？比如监督学习、无监督学习、深度学习，还是其他具体的应用场景？我们可以从任何你感兴趣的话题开始讨论！

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent1 (to agent2):

你好！很高兴你对机器学习感兴趣。机器学习确实是一个广泛且充满挑战的领域。你提到的监督学习、无监督学习和深度学习都是非常重要的方向。此外，强化学习、自然语言处理、计算机视觉等也是热门的研究和应用领域。

如果你对某个具体的方向感兴趣，我们可以深入讨论。例如，监督学习主要关注如何利用标注数据来训练模型，而无监督学习则试图从未标注的数据中发现模式。深度学习通过多层神经网络来处理复杂的任务，如图像识别和语音识别。

你对哪个方向感兴趣呢？或者你有其他具体的问题或应用场景想要探讨吗？

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1):

很高兴你对机器学习有如此深入的了解！你提到的监督学习、无监督学习、深度学习、强化学习、自然语言处理和计算机视觉都是非常有趣且具有挑战性的领域。

如果你对某个具体的方向感兴趣，我们可以深入讨论。例如，监督学习主要关注如何利用标注数据来训练模型，而无监督学习则试图从未标注的数据中发现模式。深度学习通过多层神经网络来处理复杂的任务，如图像识别和语音识别。

你对哪个方向感兴

&emsp;&emsp;第二种方式，则是可以通过`carryover` 参数用于在启动新的聊天会话时传递之前对话的上下文信息。它可以是一个字符串或字符串列表，系统会将这些信息与初始消息结合起来，作为对话的上下文。其源码定义如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501071809166.png" width=100%></div>

In [13]:
# 启动一个对话
response = agent1.initiate_chat(
    recipient=agent2,
    message={
        'role': 'user', 
        'content': "你知道我叫什么吗？？？"
    },
    carryover="我的名字是木羽",  # 传递上下文信息
    max_turns=2,
)

print(response)  # 输出对话的响应

agent1 (to agent2):

你知道我叫什么吗？？？
Context: 
我的名字是木羽

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1):

你的名字是木羽。

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent1 (to agent2):

谢谢你的提醒！不过，你提到的“木羽”是你的名字，而不是我的名字。我是一个AI助手，没有具体的名字，但你可以叫我任何你喜欢的名字。如果你有其他问题或需要帮助，随时告诉我！

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1):

不客气！很高兴能为你提供帮助。如果你有任何问题或需要进一步的帮助，请随时告诉我。你可以叫我任何你喜欢的名字，或者直接叫我AI助手。期待为你解答更多问题！😊

--------------------------------------------------------------------------------
ChatResult(chat_id=None, chat_history=[{'content': '你知道我叫什么吗？？？\nContext: \n我的名字是木羽', 'role': 'assistant', 'name': 'agent1'}, {'content': '你的名字是木羽。', 'role': 'user', 'name': 'agent2'}, {'content': '谢谢你的提醒！不过，你提到的“木羽”是你的名字，而不是我的名字。我是一个AI助手，没有具体的名字，但你可以叫我任何你喜欢的名字。如果你有其他问题或需要帮助，随时告诉我！', 'role': 'as

&emsp;&emsp;自定义 `Callable` 是一个可调用的函数或对象，可以作为 `initiate_chat` 方法的 `message` 参数传递，用于根据传入的上下文信息（如发送方、接收方和上下文）动态生成定制化的初始消息，适用于需要基于复杂逻辑、动态数据或特定条件生成消息的场景。另一方面，`carryover` 参数用于在启动新的聊天会话时传递之前对话的上下文信息，可以是字符串或字符串列表，代表之前的对话内容或关键信息，适用于在新会话中延续或参考之前对话内容，以保持上下文的连贯性。

## 1.2 summarizer 参数解读

&emsp;&emsp;在 `initiate_chat` 方法中的 `Summarizer` 的作用是生成对话的总结，借助`summary_method` 参数可以使用三种不同的摘要生成方法，分别是：

- **`"last_msg"`**：返回对话中的最后一条消息，作为总结。
- **`"reflection_with_llm"`**：使用 LLM（大语言模型）对对话进行反思，提取总结。这种方法能够生成更加语义丰富的摘要，基于对话的上下文。
- **自定义摘要方法**：还可以定义自己的摘要生成函数，通过传入 `summary_args` 来进一步定制摘要的生成方式。

&emsp;&emsp; `AutoGen` 中的 `Summarizer` 是通过 `summary_method` 参数进行配置的，默认是使用`last_msg`，其源码定义如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081035271.png" width=100%></div>

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081036760.png" width=100%></div>

&emsp;&emsp;因此，当我们不进行`summary_method`的指定时，则如下代码所示：

In [14]:
# 定义了一个ConversableAgent实例
agent1 = ConversableAgent(
    "agent1",
    llm_config={"config_list": config_model},  # 这里使用 config_model
)

agent2 = ConversableAgent(
    "agent2",
    llm_config={"config_list": config_model},  # 这里使用 config_model
)

# 启动一个对话
response = agent1.initiate_chat(
    recipient=agent2,
    message="什么是黑洞？你能用通俗的语言解释吗？",
    max_turns=2,
)

print(response)  # 输出对话的响应

agent1 (to agent2):

什么是黑洞？你能用通俗的语言解释吗？

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1):

当然可以！黑洞是宇宙中一种非常神秘的天体，它的引力极其强大，强大到连光都无法逃脱。你可以把黑洞想象成一个“宇宙吸尘器”，任何靠近它的东西，包括光，都会被吸进去，再也出不来。

黑洞的形成通常是因为一颗非常大的恒星在生命末期发生了剧烈的爆炸（超新星爆炸），然后它的核心坍缩成一个极其致密的点，这个点的引力非常强，以至于在它周围形成了一个“事件视界”。一旦任何东西跨过这个“事件视界”，就再也无法逃脱了。

简单来说，黑洞就像一个“看不见的陷阱”，你无法直接看到它，但可以通过它对周围物质和光的影响来推断它的存在。科学家们通过观察黑洞周围的物质如何运动，以及它如何扭曲光线，来研究这些神秘的天体。

希望这个解释能让你对黑洞有个初步的了解！

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent1 (to agent2):

哇，黑洞听起来真的很神奇！那黑洞会不会把整个宇宙都吸进去呢？这是个有趣的问题，因为黑洞虽然引力很强，但它并不是无限制地吞噬一切。黑洞的引力范围和它的质量有关，通常只影响它附近的天体或物质。比如，我们银河系的中心就有一个超大质量黑洞，但它并没有把整个银河系吸进去，因为银河系中的恒星和其他物质都在围绕它运动，就像地球围绕太阳转一样。

所以，黑洞虽然强大，但它的影响范围是有限的，不会把整个宇宙都吸进去。宇宙中有很多黑洞，但它们只是宇宙的一部分，而不是主宰者。希望这个解释能让你更清楚一些！

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1)

&emsp;&emsp;这里看最终的结果：

In [15]:
response

ChatResult(chat_id=None, chat_history=[{'content': '什么是黑洞？你能用通俗的语言解释吗？', 'role': 'assistant', 'name': 'agent1'}, {'content': '当然可以！黑洞是宇宙中一种非常神秘的天体，它的引力极其强大，强大到连光都无法逃脱。你可以把黑洞想象成一个“宇宙吸尘器”，任何靠近它的东西，包括光，都会被吸进去，再也出不来。\n\n黑洞的形成通常是因为一颗非常大的恒星在生命末期发生了剧烈的爆炸（超新星爆炸），然后它的核心坍缩成一个极其致密的点，这个点的引力非常强，以至于在它周围形成了一个“事件视界”。一旦任何东西跨过这个“事件视界”，就再也无法逃脱了。\n\n简单来说，黑洞就像一个“看不见的陷阱”，你无法直接看到它，但可以通过它对周围物质和光的影响来推断它的存在。科学家们通过观察黑洞周围的物质如何运动，以及它如何扭曲光线，来研究这些神秘的天体。\n\n希望这个解释能让你对黑洞有个初步的了解！', 'role': 'user', 'name': 'agent2'}, {'content': '哇，黑洞听起来真的很神奇！那黑洞会不会把整个宇宙都吸进去呢？这是个有趣的问题，因为黑洞虽然引力很强，但它并不是无限制地吞噬一切。黑洞的引力范围和它的质量有关，通常只影响它附近的天体或物质。比如，我们银河系的中心就有一个超大质量黑洞，但它并没有把整个银河系吸进去，因为银河系中的恒星和其他物质都在围绕它运动，就像地球围绕太阳转一样。\n\n所以，黑洞虽然强大，但它的影响范围是有限的，不会把整个宇宙都吸进去。宇宙中有很多黑洞，但它们只是宇宙的一部分，而不是主宰者。希望这个解释能让你更清楚一些！', 'role': 'assistant', 'name': 'agent1'}, {'content': '不用担心，黑洞不会把整个宇宙都吸进去！虽然黑洞的引力非常强大，但它的影响范围是有限的。每个黑洞都有一个“事件视界”，也就是一个边界，只有跨过这个边界的物质才会被吸进去。在事件视界之外，黑洞的引力和普通天体的引力差不多。\n\n举个例子，我们银河系的中心有一个超大质量黑洞，但它并没有把整个银河系吸进去。银河系中的恒星、行星和其他物质都在围绕这个黑洞运动，就像地球围绕太阳转一样。黑洞只是宇宙中的

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081041029.png" width=100%></div>

&emsp;&emsp;`summary`中的内容将会作为此轮对话的最终结果输出给用户端。

In [16]:
response.summary

'不用担心，黑洞不会把整个宇宙都吸进去！虽然黑洞的引力非常强大，但它的影响范围是有限的。每个黑洞都有一个“事件视界”，也就是一个边界，只有跨过这个边界的物质才会被吸进去。在事件视界之外，黑洞的引力和普通天体的引力差不多。\n\n举个例子，我们银河系的中心有一个超大质量黑洞，但它并没有把整个银河系吸进去。银河系中的恒星、行星和其他物质都在围绕这个黑洞运动，就像地球围绕太阳转一样。黑洞只是宇宙中的一部分，它们的存在并不会导致整个宇宙被吞噬。\n\n所以，黑洞虽然很强大，但它们的影响范围是有限的，不会对宇宙的整体结构造成威胁。希望这个解释能让你更放心一些！'

&emsp;&emsp;除此以外，`summary_method`也可以设置为`reflection_with_llm`，它会从`chat_history`中获取到对话过程中产生的消息列表，并使用大模型来进行总结。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081049045.png" width=100%></div>

&emsp;&emsp;具体的执行逻辑是：首先尝试使用接收`Agent`定义的大模型实例 ，如果不可用，则使用发送`Agent`定义的大模型实例。而默认的提示如下定义所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081052407.png" width=100%></div>

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081052406.png" width=100%></div>

&emsp;&emsp;因此，如果想让大模型进行总结的话，则代码如下所示：

In [17]:
# 定义了一个ConversableAgent实例
agent1 = ConversableAgent(
    "agent1",
    llm_config={"config_list": config_model},  # 这里使用 config_model
)

agent2 = ConversableAgent(
    "agent2",
    llm_config={"config_list": config_model},  # 这里使用 config_model
)

# 启动一个对话
response = agent1.initiate_chat(
    recipient=agent2,
    max_turns=2,
    summary_method="reflection_with_llm",
    message="什么是黑洞？你能用通俗的语言解释吗？",
)

print(response)  # 输出对话的响应

agent1 (to agent2):

什么是黑洞？你能用通俗的语言解释吗？

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1):

当然可以！黑洞是宇宙中一种非常神秘的天体，它的引力极其强大，强大到连光都无法逃脱。你可以把黑洞想象成一个“宇宙吸尘器”，任何靠近它的东西，包括光，都会被吸进去，再也出不来。

黑洞的形成通常是因为一颗非常大的恒星在生命末期发生了剧烈的爆炸（超新星爆炸），然后它的核心坍缩成一个极其致密的点，这个点的引力非常强，以至于在它周围形成了一个“事件视界”。一旦任何东西跨过这个“事件视界”，就再也无法逃脱了。

简单来说，黑洞就像一个“看不见的陷阱”，你无法直接看到它，但可以通过它对周围物质和光的影响来推断它的存在。科学家们通过观察黑洞周围的物质如何运动，以及它如何扭曲光线，来研究这些神秘的天体。

希望这个解释能让你对黑洞有个初步的了解！

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent1 (to agent2):

哇，黑洞听起来真的很神奇！那黑洞会不会把整个宇宙都吸进去呢？这是个有趣的问题，因为黑洞虽然引力很强，但它并不是无限制地吞噬一切。黑洞的引力范围和它的质量有关，通常只影响它附近的天体或物质。比如，我们银河系的中心就有一个超大质量黑洞，但它并没有把整个银河系吸进去，因为银河系中的恒星和其他物质都在围绕它运动，就像地球围绕太阳转一样。

所以，黑洞虽然强大，但它的影响范围是有限的，不会把整个宇宙都吸进去。宇宙中有很多黑洞，但它们只是宇宙的一部分，而不是主宰者。希望这个解释能让你更清楚一些！

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1)

In [18]:
response.summary

'黑洞是宇宙中一种极其致密的天体，引力强大到连光都无法逃脱。它通常由大质量恒星在生命末期坍缩形成，核心压缩成一个密度极高的点，周围形成一个“事件视界”，任何跨过这个边界的物质都无法逃脱。虽然黑洞引力极强，但它的影响范围有限，不会无限制地吞噬一切。例如，银河系中心的超大质量黑洞并未吞噬整个星系，星系中的天体仍在围绕它运动。因此，黑洞不会吞噬整个宇宙，它们只是宇宙结构的一部分。'

In [19]:
import pprint

pprint.pprint(response.chat_history)

[{'content': '什么是黑洞？你能用通俗的语言解释吗？', 'name': 'agent1', 'role': 'assistant'},
 {'content': '当然可以！黑洞是宇宙中一种非常神秘的天体，它的引力极其强大，强大到连光都无法逃脱。你可以把黑洞想象成一个“宇宙吸尘器”，任何靠近它的东西，包括光，都会被吸进去，再也出不来。\n'
             '\n'
             '黑洞的形成通常是因为一颗非常大的恒星在生命末期发生了剧烈的爆炸（超新星爆炸），然后它的核心坍缩成一个极其致密的点，这个点的引力非常强，以至于在它周围形成了一个“事件视界”。一旦任何东西跨过这个“事件视界”，就再也无法逃脱了。\n'
             '\n'
             '简单来说，黑洞就像一个“看不见的陷阱”，你无法直接看到它，但可以通过它对周围物质和光的影响来推断它的存在。科学家们通过观察黑洞周围的物质如何运动，以及它如何扭曲光线，来研究这些神秘的天体。\n'
             '\n'
             '希望这个解释能让你对黑洞有个初步的了解！',
  'name': 'agent2',
  'role': 'user'},
 {'content': '哇，黑洞听起来真的很神奇！那黑洞会不会把整个宇宙都吸进去呢？这是个有趣的问题，因为黑洞虽然引力很强，但它并不是无限制地吞噬一切。黑洞的引力范围和它的质量有关，通常只影响它附近的天体或物质。比如，我们银河系的中心就有一个超大质量黑洞，但它并没有把整个银河系吸进去，因为银河系中的恒星和其他物质都在围绕它运动，就像地球围绕太阳转一样。\n'
             '\n'
             '所以，黑洞虽然强大，但它的影响范围是有限的，不会把整个宇宙都吸进去。宇宙中有很多黑洞，但它们只是宇宙的一部分，而不是主宰者。希望这个解释能让你更清楚一些！',
  'name': 'agent1',
  'role': 'assistant'},
 {'content': '不用担心，黑洞不会把整个宇宙都吸进去！虽然黑洞的引力非常强大，但它的影响范围是有限的。每个黑洞都有一个“事件视界”，也就是一个边界，只有跨过这个边界的物质才会被吸进去。在事件视界之外，黑洞的引

&emsp;&emsp;这里就可以看到，当使用是 `reflection_with_llm` 方法，会在对话过程中逐步提取要点，帮助用户理解对话的核心内容，特别是在长时间或复杂的对话中。通过输出清晰的摘要，可以快速回顾对话的要点，而不需要再次浏览整个对话内容。

&emsp;&emsp;当然，第三种方法我们也可以自定义大模型的摘要过程。当`summary_method`参数指定为`reflection_with_llm`时，搭配`summary_args`参数可以允许我们传递自定义的参数，以便进一步定制摘要生成的行为，比如提供摘要时的提示词（prompt）或者指定摘要的角色。其中：

- summary_prompt：指定用于提示大模型模型生成摘要的文本内容。例如，你可以自定义提示模型如何总结对话，是否应该强调某些关键点。
- summary_role：指定摘要生成时使用的角色，可以选择`system`或者`user`，不指定的话默认是`system`


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081103131.png" width=100%></div>

In [22]:
# 定义一个ConversableAgent实例
agent1 = ConversableAgent(
    "agent1",
    llm_config={"config_list": config_model},  # 这里使用 config_model
)

agent2 = ConversableAgent(
    "agent2",
    llm_config={"config_list": config_model},  # 这里使用 config_model
)

# 启动一个对话
response = agent1.initiate_chat(
    recipient=agent2,
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_args = {
        "summary_role": "system",  # 设置角色为 system
        "summary_prompt": "总结这次谈话中讨论的主要问题和提出的解决办法。关注技术细节和解决方案。。"  # 这里添加摘要的提示词
    },
    message="如何解决全球气候变暖的问题？",
)

print(response)  # 输出对话的响应

agent1 (to agent2):

如何解决全球气候变暖的问题？

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1):

全球气候变暖是一个复杂且紧迫的问题，需要全球范围内的共同努力和多元化的解决方案。以下是一些关键的解决措施：

### 1. **减少温室气体排放**
   - **能源转型**：加速从化石燃料（如煤、石油、天然气）向可再生能源（如太阳能、风能、水能）的转变。
   - **提高能效**：推广节能技术和设备，减少能源消耗。
   - **交通减排**：推广电动汽车、公共交通和非机动交通工具，减少交通运输中的碳排放。
   - **工业减排**：改进工业生产流程，采用低碳技术，减少工业排放。

### 2. **保护和恢复生态系统**
   - **植树造林**：森林是重要的碳汇，植树造林和防止森林砍伐有助于吸收大气中的二氧化碳。
   - **保护海洋**：海洋也是重要的碳汇，保护海洋生态系统（如珊瑚礁、红树林）有助于维持地球的碳平衡。
   - **恢复湿地**：湿地能够储存大量碳，恢复和保护湿地有助于减少温室气体排放。

### 3. **推动可持续农业**
   - **减少农业排放**：改进农业实践，减少甲烷和一氧化二氮的排放。
   - **推广有机农业**：减少化肥和农药的使用，推广可持续的农业方法。
   - **减少食物浪费**：食物浪费会导致不必要的温室气体排放，减少浪费有助于降低碳足迹。

### 4. **加强国际合作**
   - **履行《巴黎协定》**：各国应履行其在《巴黎协定》中的承诺，共同努力将全球气温升幅控制在1.5°C以内。
   - **技术转让与资金支持**：发达国家应向发展中国家提供技术和资金支持，帮助其实现低碳发展。

### 5. **提高公众意识与行为改变**
   - **教育与宣传**：通过教育和宣传提高公众对气候变化的认识，鼓励个人和社区采取行动。
   - **绿色消费**：鼓励消费者选择低碳产品和服务，减少碳足迹。
   - **减少浪费**：推广循环经济，减少资源浪费和环境污染

In [24]:
print(response.summary)

你的总结非常全面且系统化，涵盖了从政策到技术、从国际到个人的多层次解决方案。以下是对你提到的关键措施的进一步技术细节和具体实施路径的补充，以帮助更深入地理解如何应对全球气候变暖问题：

---

### 1. **减少温室气体排放**
   - **能源转型**：
     - **可再生能源技术**：太阳能光伏（PV）和风能是目前最成熟的可再生能源技术。太阳能光伏的效率正在不断提高，而风能技术也在向更大规模和更高效率发展。
     - **储能技术**：电池储能（如锂离子电池）和抽水蓄能是解决可再生能源间歇性问题的关键技术。新型储能技术（如固态电池、液流电池）也在研发中。
   - **提高能效**：
     - **智能电网**：通过智能电表和实时数据分析优化电力分配，减少能源浪费。
     - **建筑能效**：使用高效隔热材料、智能温控系统和LED照明等技术，显著降低建筑能耗。
   - **交通减排**：
     - **电动汽车（EV）**：电池技术的进步（如固态电池）正在提高电动汽车的续航里程和充电速度。
     - **氢燃料电池**：氢能作为一种清洁能源，正在被用于重型运输（如卡车、船舶）和工业领域。
   - **工业减排**：
     - **碳捕集与利用（CCU）**：将工业排放的二氧化碳转化为有用的化学品或燃料，如甲醇或合成燃料。
     - **电气化工艺**：在钢铁、水泥等高耗能行业中推广电炉和电加热技术。

---

### 2. **保护和恢复生态系统**
   - **植树造林**：
     - **无人机植树**：利用无人机进行大规模植树，提高效率和覆盖率。
     - **森林监测**：通过卫星遥感和人工智能技术实时监测森林健康状况。
   - **保护海洋**：
     - **海洋碳汇**：通过人工上升流技术增强海洋表层的营养供应，促进浮游植物生长，增加碳吸收。
     - **珊瑚礁修复**：利用3D打印技术制造人工珊瑚礁，促进海洋生物多样性恢复。
   - **恢复湿地**：
     - **湿地碳汇监测**：利用传感器网络和遥感技术监测湿地的碳储存能力。
     - **生态修复**：通过引入本地植物和动物物种，恢复湿地的生态功能。

---

### 3. **推动可持续农业**
   -

# 2. 如何控制代理对话的终止

&emsp;&emsp;`AutoGen`框架整体是构架对话式多代理架构来完成`Mulit-Agent`的系统，通过让多个代理进行自主对话来协作完成复杂任务，那么如何控制对话的终止则是非常重要的一个环节，如果没有合理的终止机制，对话会无休止地进行下去，因此也针对不同的场景，抽象出了多种不同的结束对话的方法，比如在`initiate_chat`方法中的`max_turns`, 指定对话可以进行的最大轮次（turns）。一轮对话是指发送一个消息和接收一个回应。设置了 max_turns 后，达到这个限制时，对话会自动结束。

&emsp;&emsp;除此之外，`AutoGen` 中的 `Conversable` 类也提供了多种方法和参数来终止对话。这些参数用于控制对话是否会在特定条件下结束或终止。主要包括：

- **is_termination_msg**

&emsp;&emsp;如果满足特定的条件则触发终止。例如，在对话结束时，用户可能会发送“结束”或“退出”之类的消息，代理通过该函数判断并及时结束对话，避免无意义的继续互动。如下代码示例：

In [29]:
# 创建两个代理：数学专家和数据分析师
math_expert = ConversableAgent(
    name="math_expert",    # 数学专家
    system_message="你是数学专家，将帮助解决统计问题。",
    llm_config={"config_list": config_model},  # 这里使用 config_model
    is_termination_msg=lambda msg: "22" in msg["content"],  # 如果回答是22，结束对话
)

data_analyst = ConversableAgent(
    name="data_analyst",  # 数据分析师
    system_message="你是一名数据分析师。你的工作是验证和确认对数学问题的分析。",
    llm_config={"config_list": config_model},  # 这里使用 config_model
)

# 启动对话：数学专家提出问题
result = math_expert.initiate_chat(
    recipient=data_analyst,
    message="总共有30名学生。18人通过了数学考试，12人通过了英语考试，8人通过了两门考试。有多少学生通过了数学或英语考试？？",
    summary_method="reflection_with_llm",
    summary_args = {
        "summary_role": "system",  # 设置角色为 system
        "summary_prompt": "请仅返回计算出来的最终结果，不要输出任何与结果无关的内容，一定要简洁。"  # 这里添加摘要的提示词
    },
)

# 输出结果
print(result)

math_expert (to data_analyst):

总共有30名学生。18人通过了数学考试，12人通过了英语考试，8人通过了两门考试。有多少学生通过了数学或英语考试？？

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
data_analyst (to math_expert):

要计算通过数学或英语考试的学生人数，我们可以使用容斥原理。容斥原理公式如下：

\[ |A \cup B| = |A| + |B| - |A \cap B| \]

其中：
- \( |A| \) 是通过数学考试的学生人数，即18人。
- \( |B| \) 是通过英语考试的学生人数，即12人。
- \( |A \cap B| \) 是同时通过两门考试的学生人数，即8人。

将数值代入公式：

\[ |A \cup B| = 18 + 12 - 8 = 22 \]

因此，通过数学或英语考试的学生人数是22人。

--------------------------------------------------------------------------------


Please give feedback to data_analyst. Press enter or type 'exit' to stop the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': '总共有30名学生。18人通过了数学考试，12人通过了英语考试，8人通过了两门考试。有多少学生通过了数学或英语考试？？', 'role': 'assistant', 'name': 'math_expert'}, {'content': '要计算通过数学或英语考试的学生人数，我们可以使用容斥原理。容斥原理公式如下：\n\n\\[ |A \\cup B| = |A| + |B| - |A \\cap B| \\]\n\n其中：\n- \\( |A| \\) 是通过数学考试的学生人数，即18人。\n- \\( |B| \\) 是通过英语考试的学生人数，即12人。\n- \\( |A \\cap B| \\) 是同时通过两门考试的学生人数，即8人。\n\n将数值代入公式：\n\n\\[ |A \\cup B| = 18 + 12 - 8 = 22 \\]\n\n因此，通过数学或英语考试的学生人数是22人。', 'role': 'user', 'name': 'data_analyst'}], summary='22', cost={'usage_including_cached_inference': {'total_cost': 0.0, 'deepseek-chat': {'cost': 0.0, 'prompt_tokens': 250, 'completion_tokens': 145, 'total_tokens': 395}}, 'usage_excluding_cached_inference': {'total_cost': 0.0, 'deepseek-chat': {'cost': 0.0, 'prompt_tokens': 250, 'completion_tokens': 145, 'total_tokens': 395}}}, human_input=['exit'])


In [30]:
result.summary

'22'

&emsp;&emsp;这里可以看到返回的结果则是非常精准的计算结果。大家可以按照这种思路进行尝试。

- **max_consecutive_auto_reply**

&emsp;&emsp;如果对同一发送方`Agent`的自动回复数量超过设置的临界点，则会终止。代码如下：

In [31]:
# 定义一个ConversableAgent实例
agent1 = ConversableAgent(
    "agent1",
    llm_config={"config_list": config_model},  # 这里使用 config_model
    max_consecutive_auto_reply=1
)

agent2 = ConversableAgent(
    "agent2",
    llm_config={"config_list": config_model},  # 这里使用 config_model
)

# 启动一个对话
response = agent1.initiate_chat(
    recipient=agent2,
    message="如何解决全球气候变暖的问题？",
)

print(response)  # 输出对话的响应

agent1 (to agent2):

如何解决全球气候变暖的问题？

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent2 (to agent1):

全球气候变暖是一个复杂且紧迫的问题，需要全球范围内的共同努力和多元化的解决方案。以下是一些关键的解决措施：

### 1. **减少温室气体排放**
   - **能源转型**：加速从化石燃料（如煤、石油、天然气）向可再生能源（如太阳能、风能、水能）的转变。
   - **提高能效**：推广节能技术和设备，减少能源消耗。
   - **交通减排**：推广电动汽车、公共交通和非机动交通工具，减少交通运输中的碳排放。
   - **工业减排**：改进工业生产流程，采用低碳技术，减少工业排放。

### 2. **保护和恢复生态系统**
   - **植树造林**：森林是重要的碳汇，植树造林和防止森林砍伐有助于吸收大气中的二氧化碳。
   - **保护海洋**：海洋也是重要的碳汇，保护海洋生态系统（如珊瑚礁、红树林）有助于维持地球的碳平衡。
   - **恢复湿地**：湿地能够储存大量碳，恢复和保护湿地有助于减少温室气体排放。

### 3. **推动可持续农业**
   - **减少农业排放**：改进农业实践，减少甲烷和一氧化二氮的排放。
   - **推广有机农业**：减少化肥和农药的使用，推广可持续的农业方法。
   - **减少食物浪费**：食物浪费会导致不必要的温室气体排放，减少浪费有助于降低碳足迹。

### 4. **加强国际合作**
   - **履行《巴黎协定》**：各国应履行其在《巴黎协定》中的承诺，共同努力将全球气温升幅控制在1.5°C以内。
   - **技术转让与资金支持**：发达国家应向发展中国家提供技术和资金支持，帮助其实现低碳发展。

### 5. **提高公众意识与行为改变**
   - **教育与宣传**：通过教育和宣传提高公众对气候变化的认识，鼓励个人和社区采取行动。
   - **绿色消费**：鼓励消费者选择低碳产品和服务，减少碳足迹。
   - **减少浪费**：推广循环经济，减少资源浪费和环境污染

Please give feedback to agent2. Press enter to skip and use auto-reply, or type 'exit' to stop the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': '如何解决全球气候变暖的问题？', 'role': 'assistant', 'name': 'agent1'}, {'content': '全球气候变暖是一个复杂且紧迫的问题，需要全球范围内的共同努力和多元化的解决方案。以下是一些关键的解决措施：\n\n### 1. **减少温室气体排放**\n   - **能源转型**：加速从化石燃料（如煤、石油、天然气）向可再生能源（如太阳能、风能、水能）的转变。\n   - **提高能效**：推广节能技术和设备，减少能源消耗。\n   - **交通减排**：推广电动汽车、公共交通和非机动交通工具，减少交通运输中的碳排放。\n   - **工业减排**：改进工业生产流程，采用低碳技术，减少工业排放。\n\n### 2. **保护和恢复生态系统**\n   - **植树造林**：森林是重要的碳汇，植树造林和防止森林砍伐有助于吸收大气中的二氧化碳。\n   - **保护海洋**：海洋也是重要的碳汇，保护海洋生态系统（如珊瑚礁、红树林）有助于维持地球的碳平衡。\n   - **恢复湿地**：湿地能够储存大量碳，恢复和保护湿地有助于减少温室气体排放。\n\n### 3. **推动可持续农业**\n   - **减少农业排放**：改进农业实践，减少甲烷和一氧化二氮的排放。\n   - **推广有机农业**：减少化肥和农药的使用，推广可持续的农业方法。\n   - **减少食物浪费**：食物浪费会导致不必要的温室气体排放，减少浪费有助于降低碳足迹。\n\n### 4. **加强国际合作**\n   - **履行《巴黎协定》**：各国应履行其在《巴黎协定》中的承诺，共同努力将全球气温升幅控制在1.5°C以内。\n   - **技术转让与资金支持**：发达国家应向发展中国家提供技术和资金支持，帮助其实现低碳发展。\n\n### 5. **提高公众意识与行为改变**\n   - **教育与宣传**：通过教育和宣传提高公众对气候变化的认识，鼓励个人和社区采取行动。\n   - **绿色消费**：鼓励消费者选择低碳产品和服务，减少碳足迹。\n   - **减少浪费**：推广循环经济，减少资源浪费和环境污染。\n\n### 6. **技术创新**\n

&emsp;&emsp;那么在上述两个终止的示例中，当触发了终止的条件，并不会直接结束当前的对话，还是弹出一个`input`输入框，需要手动输入`exit`才能终止，这个是什么原因呢？

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081146578.png" width=100%></div>

&emsp;&emsp;我们来看下源码理解其原理：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081151647.png" width=100%></div>

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081151648.png" width=100%></div>

&emsp;&emsp;这是因为 `is_termination_msg`、`max_consecutive_auto_reply`的参数功能本质上是用来检查对话中的特定条件，并标记是否应该终止对话。然而，它并不会直接触发对话的停止，而只是通过判断收到的消息是否符合终止条件来确定是否应该终止。我们遇到当触发了终止条件后还需要手动输入 exit 才会结束对话的问题是因为**在 `human_input_mode` 不是 `NEVER` 或 `TERMINATE` 时，系统并不会主动停止对话，而是期望用户明确操作来结束对话。**

&emsp;&emsp;因此，引出了我们需要掌握的另一个知识点：`AutoGen`中的`Human in the loop`。

# 3. 人机交互（Human in the loop）

&emsp;&emsp;在`Agent`应用系统开发过程中，往往由大模型负责大部分的处理和决策，但在某些复杂、模糊或高风险的场景下，一般请求人工干预来提供判断、修改或确认。**"Human in the loop"**（HIL）是指在对话过程中，某些时刻需要人工干预或输入来完成任务或进行决策。基本上成熟的`Agent`开发框架都会设计人工交互的逻辑。简而言之，它是一种设计模式，其中`Agent`和人类共同参与决策或执行任务的过程。

&emsp;&emsp;在 `AutoGen` 的 `ConversableAgent` 中，`human_input_mode` 就是控制是否启用“Human in the loop”的核心参数。这个参数决定了是否在每次消息接收时请求人工输入，以及在何种情况下终止对话。`human_input_mode` 主要有以下几种设置：

1. **"NEVER"**：
   - 代理完全不需要人工输入。所有决策和回复都由代理自动完成，不需要人工干预。这种模式适用于较为简单的任务或自动化流程。
   - 示例：在标准的问答系统或自动化对话中，代理会自动回复问题，并且不会主动请求人工输入。
<br>
<br>
2. **"ALWAYS"**：
   - 代理会在每轮对话中都请求人工输入。这意味着系统的每个回应都可能需要人工确认或补充才能继续进行。
   - 示例：当代理进行复杂推理时，系统可能会将计算结果或推理步骤呈现给人工来确认是否正确。
<br>
<br>
3. **"TERMINATE"**：
   - 这种模式下，代理仅在满足特定条件时（如终止消息或连续自动回复次数超过最大值时）请求人工输入。代理在大多数情况下会自动完成对话，但在满足条件时会请求人工干预来决定是否继续或结束对话。
   - 示例：如果在对话过程中代理无法确定某个问题的答案，或者有较高风险的决策需要人工确认时，系统会询问人工以继续对话。

&emsp;&emsp;在到具体的流程设计中，人机交互组件位于自动回复组件之前。它可以拦截传入的消息并决定是将其传递给自动回复组件还是提供人工反馈。如下图所示：

> 图片来源于 AutoGen 官方文档：https://microsoft.github.io/autogen/0.2/docs/tutorial/human-in-the-loop

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501071231497.png" width=100%></div>

&emsp;&emsp;我们就依次实践一下这三种不同的模式，首先来看`ALWAYS`。

- **NRVER**

&emsp;&emsp;在此模式下，从不请求人工输入，并且使用终止条件来终止。所以当我们希望代理完全自主行动时，此模式非常有用。比如如下代码：

In [34]:
# 创建两个代理：数学专家和数据分析师
math_expert = ConversableAgent(
    name="math_expert",    # 数学专家
    system_message="你是数学专家，将帮助解决统计问题。",
    llm_config={"config_list": config_model},  # 这里使用 config_model
    is_termination_msg=lambda msg: "22" in msg["content"],  # 如果回答是22，结束对话
    human_input_mode="NEVER",  # 不允许人工介入

)

data_analyst = ConversableAgent(
    name="data_analyst",  # 数据分析师
    system_message="你是一名数据分析师。你的工作是验证和确认对数学问题的分析。",
    llm_config={"config_list": config_model},  # 这里使用 config_model
    human_input_mode="NEVER",  # 不允许人工介入
)

# 启动对话：数学专家提出问题
result = math_expert.initiate_chat(
    recipient=data_analyst,
    message="总共有30名学生。18人通过了数学考试，12人通过了英语考试，8人通过了两门考试。有多少学生通过了数学或英语考试？？",
    summary_method="reflection_with_llm",
    summary_args = {
        "summary_role": "system",  # 设置角色为 system
        "summary_prompt": "请仅返回计算出来的最终结果，不要输出任何与结果无关的内容，一定要简洁。"  # 这里添加摘要的提示词
    },
)

# 输出结果
print(result)

math_expert (to data_analyst):

总共有30名学生。18人通过了数学考试，12人通过了英语考试，8人通过了两门考试。有多少学生通过了数学或英语考试？？

--------------------------------------------------------------------------------
data_analyst (to math_expert):

要计算通过数学或英语考试的学生人数，我们可以使用容斥原理。容斥原理公式如下：

\[ |A \cup B| = |A| + |B| - |A \cap B| \]

其中：
- \( |A| \) 是通过数学考试的学生人数，即18人。
- \( |B| \) 是通过英语考试的学生人数，即12人。
- \( |A \cap B| \) 是同时通过两门考试的学生人数，即8人。

将数值代入公式：

\[ |A \cup B| = 18 + 12 - 8 = 22 \]

因此，通过数学或英语考试的学生人数是22人。

--------------------------------------------------------------------------------
ChatResult(chat_id=None, chat_history=[{'content': '总共有30名学生。18人通过了数学考试，12人通过了英语考试，8人通过了两门考试。有多少学生通过了数学或英语考试？？', 'role': 'assistant', 'name': 'math_expert'}, {'content': '要计算通过数学或英语考试的学生人数，我们可以使用容斥原理。容斥原理公式如下：\n\n\\[ |A \\cup B| = |A| + |B| - |A \\cap B| \\]\n\n其中：\n- \\( |A| \\) 是通过数学考试的学生人数，即18人。\n- \\( |B| \\) 是通过英语考试的学生人数，即12人。\n- \\( |A \\cap B| \\) 是同时通过两门考试的学生人数，即8人。\n\n将数值代入公式：\n\n\\[ |A \\cup B| = 18 + 12 - 8 = 22 \\]\n\n因此，通过数学或英语考试的学生人数是22

In [35]:
print(result.summary)

22


&emsp;&emsp;这里就可以看到，当添加了`human_input_mode="NEVER"`参数后，对话代理当触发了`is_termination_msg`条件后会自动终止。

- **ALWAYS**

&emsp;&emsp;在此模式下，始终请求人工输入，并且人工可以选择跳过、拦截或终止对话。这个场景下的应用示例就非常多了，比如我们设计一个市场营销分析任务，让代理之间将逐步合作来解决实际的商业问题。具体步骤包括：

- 数学专家首先根据提供的数据进行基础的分析，计算各个广告渠道的投资回报率（ROI）。
- 数据分析师负责验证数学专家的结果，并基于分析进一步给出结论，例如哪些渠道效果最好。
- 总结阶段：在得到合理的结果后，数据分析师总结并确认是否可以终止对话

In [6]:
# 创建数学专家代理
math_expert = ConversableAgent(
    name="math_expert",  # 数学专家
    system_message="你是财务分析的数学专家，负责进行营销活动数据分析和预算计算。",
    llm_config=False,  # 这里可以使用 False，因为 human_input_mode 设置为了 人工输入，所以无需 大模型介入
    is_termination_msg=lambda msg: "再见" in msg["content"].lower(),  # 如果分析完成并说 再见，终止对话
    human_input_mode="ALWAYS",  # 每轮都需要数学专家确认结果和提出新一轮的问题
)

# 创建数据分析师代理
data_analyst = ConversableAgent(
    name="data_analyst",  # 数据分析师
    system_message="你是数据分析师，负责验证和确认数学专家的分析结果。",
    llm_config={"config_list": config_model},  # 这里使用 config_model
    human_input_mode="NEVER",  # 每轮都进行自主分析和处理，不需要人工确认
)

# 启动对话：数学专家提出预算计算问题
result = math_expert.initiate_chat(
    recipient=data_analyst,
    message="我们进行了一项市场营销活动，以下是广告渠道的投入与回报数据：\n"
            "1. 搜索广告：投入100万，回报200万\n"
            "2. 社交媒体广告：投入50万，回报120万\n"
            "3. 电视广告：投入200万，回报500万\n"
            "请计算各个广告渠道的投资回报率（ROI），并返回结果。",
    summary_method="reflection_with_llm",  # 使用 LLM 进行对话总结
    summary_args={
        "summary_role": "system",  # 设置角色为 system
        "summary_prompt": "请简洁地返回各广告渠道的投资回报率（ROI）。确保计算准确。"  # 这里添加摘要的提示词
    },
)

# 输出对话结果
print(result)

math_expert (to data_analyst):

我们进行了一项市场营销活动，以下是广告渠道的投入与回报数据：
1. 搜索广告：投入100万，回报200万
2. 社交媒体广告：投入50万，回报120万
3. 电视广告：投入200万，回报500万
请计算各个广告渠道的投资回报率（ROI），并返回结果。

--------------------------------------------------------------------------------
data_analyst (to math_expert):

投资回报率（ROI）的计算公式为：
\[ \text{ROI} = \left( \frac{\text{回报} - \text{投入}}{\text{投入}} \right) \times 100\% \]

根据提供的数据，我们可以计算每个广告渠道的ROI：

1. **搜索广告**：
   \[ \text{ROI} = \left( \frac{200万 - 100万}{100万} \right) \times 100\% = 100\% \]

2. **社交媒体广告**：
   \[ \text{ROI} = \left( \frac{120万 - 50万}{50万} \right) \times 100\% = 140\% \]

3. **电视广告**：
   \[ \text{ROI} = \left( \frac{500万 - 200万}{200万} \right) \times 100\% = 150\% \]

因此，各个广告渠道的投资回报率（ROI）如下：
- 搜索广告：100%
- 社交媒体广告：140%
- 电视广告：150%

--------------------------------------------------------------------------------


Replying as math_expert. Provide feedback to data_analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  请再次确认你的计算结果是否准确


math_expert (to data_analyst):

请再次确认你的计算结果是否准确

--------------------------------------------------------------------------------
data_analyst (to math_expert):

让我们再次仔细检查每个广告渠道的投资回报率（ROI）计算，确保准确性。

投资回报率（ROI）的计算公式为：
\[ \text{ROI} = \left( \frac{\text{回报} - \text{投入}}{\text{投入}} \right) \times 100\% \]

### 1. **搜索广告**：
- 投入：100万
- 回报：200万
\[ \text{ROI} = \left( \frac{200万 - 100万}{100万} \right) \times 100\% = 100\% \]

### 2. **社交媒体广告**：
- 投入：50万
- 回报：120万
\[ \text{ROI} = \left( \frac{120万 - 50万}{50万} \right) \times 100\% = 140\% \]

### 3. **电视广告**：
- 投入：200万
- 回报：500万
\[ \text{ROI} = \left( \frac{500万 - 200万}{200万} \right) \times 100\% = 150\% \]

经过重新计算，确认结果如下：
- **搜索广告**：100%
- **社交媒体广告**：140%
- **电视广告**：150%

计算结果准确无误。如果还有其他问题，请随时告诉我！ 😊

--------------------------------------------------------------------------------


Replying as math_expert. Provide feedback to data_analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  比较哪一个广告渠道的效果最好


math_expert (to data_analyst):

比较哪一个广告渠道的效果最好

--------------------------------------------------------------------------------
data_analyst (to math_expert):

要比较哪个广告渠道的效果最好，我们可以从**投资回报率（ROI）**和**净回报**两个角度来分析。

---

### 1. **投资回报率（ROI）比较**
ROI 表示每单位投入带来的回报比例，ROI 越高，效果越好。

- **搜索广告**：100%
- **社交媒体广告**：140%
- **电视广告**：150%

**结论**：从 ROI 来看，**电视广告**的效果最好（150%），其次是**社交媒体广告**（140%），最后是**搜索广告**（100%）。

---

### 2. **净回报比较**
净回报表示回报减去投入，净回报越高，实际收益越大。

- **搜索广告**：200万 - 100万 = **100万**
- **社交媒体广告**：120万 - 50万 = **70万**
- **电视广告**：500万 - 200万 = **300万**

**结论**：从净回报来看，**电视广告**的效果最好（净回报 300万），其次是**搜索广告**（净回报 100万），最后是**社交媒体广告**（净回报 70万）。

---

### 3. **综合比较**
- **电视广告**：ROI 最高（150%），净回报最大（300万），效果最好。
- **社交媒体广告**：ROI 较高（140%），但净回报较低（70万）。
- **搜索广告**：ROI 较低（100%），但净回报中等（100万）。

---

### 最终结论
**电视广告**的效果最好，无论是从投资回报率（ROI）还是净回报来看，它都是最优选择。如果预算充足，可以优先考虑电视广告；如果预算有限，社交媒体广告也是一个不错的选择，因为它的 ROI 较高。

--------------------------------------------------------------------------------


Replying as math_expert. Provide feedback to data_analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  电视广告的 ROI 是最高的，意味着这是最有效的广告渠道。为了提高市场营销效果，建议将电视广告的投入提高30%。你觉得如何？


math_expert (to data_analyst):

电视广告的 ROI 是最高的，意味着这是最有效的广告渠道。为了提高市场营销效果，建议将电视广告的投入提高30%。你觉得如何？

--------------------------------------------------------------------------------
data_analyst (to math_expert):

提高电视广告的投入确实可能带来更高的回报，但我们需要仔细分析这种调整的潜在影响。以下是基于数据的详细分析：

---

### 1. **当前电视广告的 ROI 和净回报**
- 当前投入：200万
- 当前回报：500万
- 当前 ROI：150%
- 当前净回报：300万

---

### 2. **提高电视广告投入 30% 后的预测**
如果将电视广告的投入提高 30%，新的投入为：
\[ 200万 \times 1.3 = 260万 \]

假设 ROI 保持不变（150%），则预测回报为：
\[ 260万 \times (1 + 1.5) = 260万 \times 2.5 = 650万 \]

预测净回报为：
\[ 650万 - 260万 = 390万 \]

---

### 3. **潜在风险与考虑**
虽然提高投入可能带来更高的回报，但需要注意以下几点：
1. **ROI 可能不会保持不变**：
   - 随着投入增加，市场可能趋于饱和，导致 ROI 下降。
   - 如果电视广告的效果已经接近峰值，进一步增加投入可能无法带来成比例的回报。

2. **预算分配问题**：
   - 提高电视广告的投入可能会挤占其他广告渠道（如搜索广告或社交媒体广告）的预算，从而影响整体营销效果。

3. **目标受众覆盖**：
   - 如果电视广告的目标受众已经接近全覆盖，增加投入可能无法显著扩大受众范围。

---

### 4. **建议**
- **小规模测试**：可以先小幅增加电视广告的投入（例如 10%-20%），观察 ROI 和净回报的变化，再决定是否进一步增加。
- **多渠道优化**：在提高电视广告投入的同时，保持对其他高 ROI 渠道（如社交媒体广告）的投入，以实现整体营销效果的最大化。
- **数据分析**：持续监控广告效果，及时

Replying as math_expert. Provide feedback to data_analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': '我们进行了一项市场营销活动，以下是广告渠道的投入与回报数据：\n1. 搜索广告：投入100万，回报200万\n2. 社交媒体广告：投入50万，回报120万\n3. 电视广告：投入200万，回报500万\n请计算各个广告渠道的投资回报率（ROI），并返回结果。', 'role': 'assistant', 'name': 'math_expert'}, {'content': '投资回报率（ROI）的计算公式为：\n\\[ \\text{ROI} = \\left( \\frac{\\text{回报} - \\text{投入}}{\\text{投入}} \\right) \\times 100\\% \\]\n\n根据提供的数据，我们可以计算每个广告渠道的ROI：\n\n1. **搜索广告**：\n   \\[ \\text{ROI} = \\left( \\frac{200万 - 100万}{100万} \\right) \\times 100\\% = 100\\% \\]\n\n2. **社交媒体广告**：\n   \\[ \\text{ROI} = \\left( \\frac{120万 - 50万}{50万} \\right) \\times 100\\% = 140\\% \\]\n\n3. **电视广告**：\n   \\[ \\text{ROI} = \\left( \\frac{500万 - 200万}{200万} \\right) \\times 100\\% = 150\\% \\]\n\n因此，各个广告渠道的投资回报率（ROI）如下：\n- 搜索广告：100%\n- 社交媒体广告：140%\n- 电视广告：150%', 'role': 'user', 'name': 'data_analyst'}, {'content': '请再次确认你的计算结果是否准确', 'role': 'assistant', 'name': 'math_expert'}, {'content': '让我们再次仔细检查每个广告渠道的投资回报率（ROI）计算，确保准确性。\n\n投资回报率（ROI）的计算公式为：\n\\[ \\text{ROI} = \\left( \

In [7]:
print(result.summary)

### 1. **当前电视广告的 ROI 和净回报**
- 当前投入：200万
- 当前回报：500万
- 当前 ROI：150%
- 当前净回报：300万

---

### 2. **提高电视广告投入 30% 后的预测**
如果将电视广告的投入提高 30%，新的投入为：
\[ 200万 \times 1.3 = 260万 \]

假设 ROI 保持不变（150%），则预测回报为：
\[ 260万 \times (1 + 1.5) = 260万 \times 2.5 = 650万 \]

预测净回报为：
\[ 650万 - 260万 = 390万 \]

---

### 3. **潜在风险与考虑**
虽然提高投入可能带来更高的回报，但需要注意以下几点：
1. **ROI 可能不会保持不变**：
   - 随着投入增加，市场可能趋于饱和，导致 ROI 下降。
   - 如果电视广告的效果已经接近峰值，进一步增加投入可能无法带来成比例的回报。

2. **预算分配问题**：
   - 提高电视广告的投入可能会挤占其他广告渠道（如搜索广告或社交媒体广告）的预算，从而影响整体营销效果。

3. **目标受众覆盖**：
   - 如果电视广告的目标受众已经接近全覆盖，增加投入可能无法显著扩大受众范围。

---

### 4. **建议**
- **小规模测试**：可以先小幅增加电视广告的投入（例如 10%-20%），观察 ROI 和净回报的变化，再决定是否进一步增加。
- **多渠道优化**：在提高电视广告投入的同时，保持对其他高 ROI 渠道（如社交媒体广告）的投入，以实现整体营销效果的最大化。
- **数据分析**：持续监控广告效果，及时调整策略，确保投入产出比最大化。

---

### 5. **总结**
提高电视广告投入 30% 可能带来更高的回报（预测净回报 390万），但需要谨慎评估 ROI 的变化和预算分配。建议先进行小规模测试，并结合多渠道优化策略，以实现最佳的市场营销效果。


&emsp;&emsp;除此外，当 `human_input_mode` 设置为 `"ALWAYS"` 时，可触发的三个行为分别是：

1. **输入内容**：如果在提示框中输入了内容，代理将使用你输入的内容作为反馈，继续进行对话。
   
2. **按下 `Enter` 跳过**：如果按下 `Enter`，则代理会使用**自动回复**（即代理的自动回应行为），而不会等待人工输入继续对话。

3. **输入 `exit` 退出对话**：如果在输入框中输入 `"exit"`，对话将结束，触发终止条件，结束当前会话。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081242349.png" width=100%></div>

&emsp;&emsp;如果用户输入了内容，`reply` 会是用户输入的内容。如果用户按 `Enter` 跳过，`reply` 就是空字符串 `""`。**`self._is_termination_msg(message)`**：这段代码会检查是否当前消息符合终止条件（例如，消息包含 `"exit"` 或其他设定的终止条件）。如果没有收到输入且消息是终止消息，`reply` 会被强制设为 `"exit"`，从而结束对话。

&emsp;&emsp;如下过程所示，当进入到人工交互的阶段，直接敲击`Enter`，则给出的提示是`NO HUMAN INPUT RECEIVED.`，且内容为空。

In [9]:
# 创建数学专家代理
math_expert = ConversableAgent(
    name="math_expert",  # 数学专家
    system_message="你是财务分析的数学专家，负责进行营销活动数据分析和预算计算。",
    llm_config=False,  # 这里可以使用 False，因为 human_input_mode 设置为了 人工输入，所以无需 大模型介入
    is_termination_msg=lambda msg: "再见" in msg["content"].lower(),  # 如果分析完成并说 再见，终止对话
    human_input_mode="ALWAYS",  # 每轮都需要数学专家确认结果和提出新一轮的问题
)

# 创建数据分析师代理
data_analyst = ConversableAgent(
    name="data_analyst",  # 数据分析师
    system_message="你是数据分析师，负责验证和确认数学专家的分析结果。",
    llm_config={"config_list": config_model},  # 这里使用 config_model
    human_input_mode="NEVER",  # 每轮都进行自主分析和处理，不需要人工确认
)

# 启动对话：数学专家提出预算计算问题
result = math_expert.initiate_chat(
    recipient=data_analyst,
    message="我们进行了一项市场营销活动，以下是广告渠道的投入与回报数据：\n"
            "1. 搜索广告：投入100万，回报200万\n"
            "2. 社交媒体广告：投入50万，回报120万\n"
            "3. 电视广告：投入200万，回报500万\n"
            "请计算各个广告渠道的投资回报率（ROI），并返回结果。",
    summary_method="reflection_with_llm",  # 使用 LLM 进行对话总结
    summary_args={
        "summary_role": "system",  # 设置角色为 system
        "summary_prompt": "请简洁地返回各广告渠道的投资回报率（ROI）。确保计算准确。"  # 这里添加摘要的提示词
    },
)

# 输出对话结果
print(result)

math_expert (to data_analyst):

我们进行了一项市场营销活动，以下是广告渠道的投入与回报数据：
1. 搜索广告：投入100万，回报200万
2. 社交媒体广告：投入50万，回报120万
3. 电视广告：投入200万，回报500万
请计算各个广告渠道的投资回报率（ROI），并返回结果。

--------------------------------------------------------------------------------
data_analyst (to math_expert):

投资回报率（ROI）的计算公式为：
\[ \text{ROI} = \left( \frac{\text{回报} - \text{投入}}{\text{投入}} \right) \times 100\% \]

根据提供的数据，我们可以计算每个广告渠道的ROI：

1. **搜索广告**：
   \[ \text{ROI} = \left( \frac{200万 - 100万}{100万} \right) \times 100\% = 100\% \]

2. **社交媒体广告**：
   \[ \text{ROI} = \left( \frac{120万 - 50万}{50万} \right) \times 100\% = 140\% \]

3. **电视广告**：
   \[ \text{ROI} = \left( \frac{500万 - 200万}{200万} \right) \times 100\% = 150\% \]

因此，各个广告渠道的投资回报率（ROI）如下：
- 搜索广告：100%
- 社交媒体广告：140%
- 电视广告：150%

--------------------------------------------------------------------------------


Replying as math_expert. Provide feedback to data_analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
math_expert (to data_analyst):



--------------------------------------------------------------------------------
data_analyst (to math_expert):

根据计算结果，以下是各个广告渠道的投资回报率（ROI）总结：

1. **搜索广告**：  
   - 投入：100万  
   - 回报：200万  
   - **ROI：100%**

2. **社交媒体广告**：  
   - 投入：50万  
   - 回报：120万  
   - **ROI：140%**

3. **电视广告**：  
   - 投入：200万  
   - 回报：500万  
   - **ROI：150%**

### 结论：
- 电视广告的ROI最高（150%），其次是社交媒体广告（140%），最后是搜索广告（100%）。  
- 从投资回报率的角度来看，电视广告和社交媒体广告的表现优于搜索广告。

--------------------------------------------------------------------------------


Replying as math_expert. Provide feedback to data_analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': '我们进行了一项市场营销活动，以下是广告渠道的投入与回报数据：\n1. 搜索广告：投入100万，回报200万\n2. 社交媒体广告：投入50万，回报120万\n3. 电视广告：投入200万，回报500万\n请计算各个广告渠道的投资回报率（ROI），并返回结果。', 'role': 'assistant', 'name': 'math_expert'}, {'content': '投资回报率（ROI）的计算公式为：\n\\[ \\text{ROI} = \\left( \\frac{\\text{回报} - \\text{投入}}{\\text{投入}} \\right) \\times 100\\% \\]\n\n根据提供的数据，我们可以计算每个广告渠道的ROI：\n\n1. **搜索广告**：\n   \\[ \\text{ROI} = \\left( \\frac{200万 - 100万}{100万} \\right) \\times 100\\% = 100\\% \\]\n\n2. **社交媒体广告**：\n   \\[ \\text{ROI} = \\left( \\frac{120万 - 50万}{50万} \\right) \\times 100\\% = 140\\% \\]\n\n3. **电视广告**：\n   \\[ \\text{ROI} = \\left( \\frac{500万 - 200万}{200万} \\right) \\times 100\\% = 150\\% \\]\n\n因此，各个广告渠道的投资回报率（ROI）如下：\n- 搜索广告：100%\n- 社交媒体广告：140%\n- 电视广告：150%', 'role': 'user', 'name': 'data_analyst'}, {'content': '', 'role': 'assistant', 'name': 'math_expert'}, {'content': '根据计算结果，以下是各个广告渠道的投资回报率（ROI）总结：\n\n1. **搜索广告**：  \n   - 投入：100万  \n   - 回报：200万  \n   - **ROI：100%**\

&emsp;&emsp;这是因为在`ConversableAgent`基类中，`default_auto_reply`就是''（空字符串）。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081246653.png" width=100%></div>

&emsp;&emsp;因此，我们也可以手动的去设置自动回复的内容，比如在定义`Agent`的时候，添加`default_auto_reply`参数。代码如下：

In [10]:
# 创建数学专家代理
math_expert = ConversableAgent(
    name="math_expert",  # 数学专家
    system_message="你是财务分析的数学专家，负责进行营销活动数据分析和预算计算。",
    llm_config=False,  # 这里可以使用 False，因为 human_input_mode 设置为了 人工输入，所以无需 大模型介入
    is_termination_msg=lambda msg: "再见" in msg["content"].lower(),  # 如果分析完成并说 再见，终止对话
    human_input_mode="ALWAYS",  # 每轮都需要数学专家确认结果和提出新一轮的问题,
    default_auto_reply="请验证你的计算结果，确保百分百的准确率。"
)

# 创建数据分析师代理
data_analyst = ConversableAgent(
    name="data_analyst",  # 数据分析师
    system_message="你是数据分析师，负责验证和确认数学专家的分析结果。",
    llm_config={"config_list": config_model},  # 这里使用 config_model
    human_input_mode="NEVER",  # 每轮都进行自主分析和处理，不需要人工确认
)

# 启动对话：数学专家提出预算计算问题
result = math_expert.initiate_chat(
    recipient=data_analyst,
    message="我们进行了一项市场营销活动，以下是广告渠道的投入与回报数据：\n"
            "1. 搜索广告：投入100万，回报200万\n"
            "2. 社交媒体广告：投入50万，回报120万\n"
            "3. 电视广告：投入200万，回报500万\n"
            "请计算各个广告渠道的投资回报率（ROI），并返回结果。",
    summary_method="reflection_with_llm",  # 使用 LLM 进行对话总结
    summary_args={
        "summary_role": "system",  # 设置角色为 system
        "summary_prompt": "请简洁地返回各广告渠道的投资回报率（ROI）。确保计算准确。"  # 这里添加摘要的提示词
    },
)

# 输出对话结果
print(result)

math_expert (to data_analyst):

我们进行了一项市场营销活动，以下是广告渠道的投入与回报数据：
1. 搜索广告：投入100万，回报200万
2. 社交媒体广告：投入50万，回报120万
3. 电视广告：投入200万，回报500万
请计算各个广告渠道的投资回报率（ROI），并返回结果。

--------------------------------------------------------------------------------
data_analyst (to math_expert):

投资回报率（ROI）的计算公式为：
\[ \text{ROI} = \left( \frac{\text{回报} - \text{投入}}{\text{投入}} \right) \times 100\% \]

根据提供的数据，我们可以计算每个广告渠道的ROI：

1. **搜索广告**：
   \[ \text{ROI} = \left( \frac{200万 - 100万}{100万} \right) \times 100\% = 100\% \]

2. **社交媒体广告**：
   \[ \text{ROI} = \left( \frac{120万 - 50万}{50万} \right) \times 100\% = 140\% \]

3. **电视广告**：
   \[ \text{ROI} = \left( \frac{500万 - 200万}{200万} \right) \times 100\% = 150\% \]

因此，各个广告渠道的投资回报率（ROI）如下：
- 搜索广告：100%
- 社交媒体广告：140%
- 电视广告：150%

--------------------------------------------------------------------------------


Replying as math_expert. Provide feedback to data_analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
math_expert (to data_analyst):

请验证你的计算结果，确保百分百的准确率。

--------------------------------------------------------------------------------
data_analyst (to math_expert):

好的，我将重新验证每个广告渠道的投资回报率（ROI）计算，确保结果的准确性。

### 计算公式：
\[ \text{ROI} = \left( \frac{\text{回报} - \text{投入}}{\text{投入}} \right) \times 100\% \]

---

### 1. **搜索广告**：
- 投入：100万
- 回报：200万

\[ \text{ROI} = \left( \frac{200万 - 100万}{100万} \right) \times 100\% = \left( \frac{100万}{100万} \right) \times 100\% = 100\% \]

**验证结果：100% 正确。**

---

### 2. **社交媒体广告**：
- 投入：50万
- 回报：120万

\[ \text{ROI} = \left( \frac{120万 - 50万}{50万} \right) \times 100\% = \left( \frac{70万}{50万} \right) \times 100\% = 140\% \]

**验证结果：140% 正确。**

---

### 3. **电视广告**：
- 投入：200万
- 回报：500万

\[ \text{ROI} = \left( \frac{500万 - 200万}{200万} \right) \times 100\% = \left( \frac{300万}{200万} \right) \times 100\% = 150\% \]

**验证结果：150% 正确。**

---

### 最终结论：
经过重新验证，所有广告渠道的 ROI 计算结果均准确无误：
- **搜索广告**：100%

Replying as math_expert. Provide feedback to data_analyst. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': '我们进行了一项市场营销活动，以下是广告渠道的投入与回报数据：\n1. 搜索广告：投入100万，回报200万\n2. 社交媒体广告：投入50万，回报120万\n3. 电视广告：投入200万，回报500万\n请计算各个广告渠道的投资回报率（ROI），并返回结果。', 'role': 'assistant', 'name': 'math_expert'}, {'content': '投资回报率（ROI）的计算公式为：\n\\[ \\text{ROI} = \\left( \\frac{\\text{回报} - \\text{投入}}{\\text{投入}} \\right) \\times 100\\% \\]\n\n根据提供的数据，我们可以计算每个广告渠道的ROI：\n\n1. **搜索广告**：\n   \\[ \\text{ROI} = \\left( \\frac{200万 - 100万}{100万} \\right) \\times 100\\% = 100\\% \\]\n\n2. **社交媒体广告**：\n   \\[ \\text{ROI} = \\left( \\frac{120万 - 50万}{50万} \\right) \\times 100\\% = 140\\% \\]\n\n3. **电视广告**：\n   \\[ \\text{ROI} = \\left( \\frac{500万 - 200万}{200万} \\right) \\times 100\\% = 150\\% \\]\n\n因此，各个广告渠道的投资回报率（ROI）如下：\n- 搜索广告：100%\n- 社交媒体广告：140%\n- 电视广告：150%', 'role': 'user', 'name': 'data_analyst'}, {'content': '请验证你的计算结果，确保百分百的准确率。', 'role': 'assistant', 'name': 'math_expert'}, {'content': '好的，我将重新验证每个广告渠道的投资回报率（ROI）计算，确保结果的准确性。\n\n### 计算公式：\n\\[ \\text{ROI} = \\left( \

&emsp;以上，就是`AutoGen`框架设计下的一个典型“人机交互”的模式，允许用户在每一轮对话后决定是否需要进行进一步的反馈或直接结束对话。

- **TERMINATE**

&emsp;&emsp;在此模式下，仅当满足终止条件时才请求人工输入。也就是说：当不触发`is_termination_msg`时，不会要求人工介入。

In [11]:
# 创建两个代理：数学专家和数据分析师
math_expert = ConversableAgent(
    name="math_expert",    # 数学专家
    system_message="你是数学专家，将帮助解决统计问题。",
    llm_config={"config_list": config_model},  # 这里使用 config_model
    is_termination_msg=lambda msg: "22" in msg["content"],  # 如果回答是22，结束对话
    human_input_mode="TERMINATE",  # 触发特定条件后人工介入

)

data_analyst = ConversableAgent(
    name="data_analyst",  # 数据分析师
    system_message="你是一名数据分析师。你的工作是验证和确认对数学问题的分析。",
    llm_config={"config_list": config_model},  # 这里使用 config_model
    human_input_mode="TERMINATE",  # 触发特定条件后人工介入
)

# 启动对话：数学专家提出问题
result = math_expert.initiate_chat(
    recipient=data_analyst,
    message="总共有30名学生。18人通过了数学考试，12人通过了英语考试，8人通过了两门考试。有多少学生通过了数学或英语考试？",
    summary_method="reflection_with_llm",
    summary_args = {
        "summary_role": "system",  # 设置角色为 system
        "summary_prompt": "请仅返回计算出来的最终结果，不要输出任何与结果无关的内容，一定要简洁。"  # 这里添加摘要的提示词
    },
)

# 输出结果
print(result)

math_expert (to data_analyst):

总共有30名学生。18人通过了数学考试，12人通过了英语考试，8人通过了两门考试。有多少学生通过了数学或英语考试？

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
data_analyst (to math_expert):

要计算通过数学或英语考试的学生总数，我们可以使用容斥原理。容斥原理公式如下：

\[ \text{通过数学或英语考试的学生数} = \text{通过数学考试的学生数} + \text{通过英语考试的学生数} - \text{同时通过两门考试的学生数} \]

根据题目给出的数据：
- 通过数学考试的学生数 = 18
- 通过英语考试的学生数 = 12
- 同时通过两门考试的学生数 = 8

将这些数值代入公式：

\[ \text{通过数学或英语考试的学生数} = 18 + 12 - 8 = 22 \]

因此，总共有22名学生通过了数学或英语考试。

--------------------------------------------------------------------------------


Please give feedback to data_analyst. Press enter or type 'exit' to stop the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': '总共有30名学生。18人通过了数学考试，12人通过了英语考试，8人通过了两门考试。有多少学生通过了数学或英语考试？', 'role': 'assistant', 'name': 'math_expert'}, {'content': '要计算通过数学或英语考试的学生总数，我们可以使用容斥原理。容斥原理公式如下：\n\n\\[ \\text{通过数学或英语考试的学生数} = \\text{通过数学考试的学生数} + \\text{通过英语考试的学生数} - \\text{同时通过两门考试的学生数} \\]\n\n根据题目给出的数据：\n- 通过数学考试的学生数 = 18\n- 通过英语考试的学生数 = 12\n- 同时通过两门考试的学生数 = 8\n\n将这些数值代入公式：\n\n\\[ \\text{通过数学或英语考试的学生数} = 18 + 12 - 8 = 22 \\]\n\n因此，总共有22名学生通过了数学或英语考试。', 'role': 'user', 'name': 'data_analyst'}], summary='22', cost={'usage_including_cached_inference': {'total_cost': 0.0, 'deepseek-chat': {'cost': 0.0, 'prompt_tokens': 257, 'completion_tokens': 152, 'total_tokens': 409}}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=['exit'])


&emsp;&emsp;所以才有了我们在第二小节实践控制代理终止时出现的情况，因为默认情况下，人工交互的模式会被设置为`TERMINATE`。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081255194.png" width=100%></div>

&emsp;&emsp;在 `ConversableAgent` 中，`human_input_mode`是实现 "Human in the loop" 的关键。通过控制这个参数的设置，系统可以决定何时依赖于自动化回复，何时需要人工干预。通过这种设计，系统可以结合自动化处理和人工灵活性，在面对不确定情况时做出更精准的决策。比如在下面这些现实场景中：

1. 任务决策：例如，在一个法律文件分析系统中，当代理发现某些段落的解释模糊或涉及到法律漏洞时，可以请求人工专家提供帮助，确保解释符合实际法律规定。

2. 异常处理：例如，在自动化的银行客服系统中，如果系统在执行交易时遇到异常（如超出限额、账户冻结等），可以让系统请求人工审核，防止错误操作。

3. 复杂决策：例如，在智能客服中，代理可以根据用户反馈生成初步解决方案，但当解决方案涉及到复杂的个人化判断时（如复杂的产品推荐），代理会请求人工干预，确保推荐方案更加精准。

&emsp;&emsp;至此，我们就掌握了`ConversableAgent`这个最关键的通用对话代理的底层构建逻辑。接下来，我们进一步探讨基于`ConversableAgent`类最具代表性的子类`AssistantAgent`和`UserProxyAgent`。

# 4. AssistantAgent和UserProxyAgent

&emsp;&emsp;`AssistantAgent` 和 `UserProxyAgent` 是 `AutoGen` 中的两个重要代理类，它们被用来模拟智能助手和用户代理的行为。

&emsp;&emsp;`AssistantAgent` 是一个以大模型为驱动的代理类，默认加载某个具体的大模型实例，不需要人工输入和代码的执行。主要用于在对话中提供大模型的推理支持、建议和解决方案。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081659661.png" width=100%></div>

```python
class AssistantAgent(ConversableAgent):

    DEFAULT_SYSTEM_MESSAGE = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply "TERMINATE" in the end when everything is done.
    """

    DEFAULT_DESCRIPTION = "A helpful and general-purpose AI assistant that has strong language skills, Python skills, and Linux command line skills."

    def __init__(
        self,
        name: str,
        system_message: Optional[str] = DEFAULT_SYSTEM_MESSAGE,
        llm_config: Optional[Union[Dict, Literal[False]]] = None,
        is_termination_msg: Optional[Callable[[Dict], bool]] = None,
        max_consecutive_auto_reply: Optional[int] = None,
        human_input_mode: Optional[str] = "NEVER",
        description: Optional[str] = None,
        **kwargs,
    ):
```

&emsp;&emsp;对应的中文系统描述如下：

```python
DEFAULT_SYSTEM_MESSAGE = """你是一个有用的人工智能助手。
用你的编程和语言技能解决任务。
在以下情况下，建议使用python代码（在python编码块中）或shell脚本（在sh编码块中）供用户执行。
1. 当您需要收集信息时，使用代码输出您需要的信息，例如，浏览或搜索网页，下载/读取文件，打印网页或文件的内容，获取当前日期/时间，检查操作系统。在打印出足够的信息，并根据你的语言能力准备好解决任务后，你就可以自己解决任务了。
2. 当您需要用代码执行某些任务时，请使用代码来执行任务并输出结果。聪明地完成任务。
如果需要的话，一步一步地解决这个任务。如果没有提供计划，首先解释你的计划。明确哪一步使用代码，哪一步使用你的语言技能。
使用代码时，必须在代码块中指示脚本类型。除了执行您建议的代码之外，用户不能提供任何其他反馈或执行任何其他操作。用户不能修改你的代码。所以不要建议用户修改不完整的代码。不要使用不打算由用户执行的代码块。
如果您希望用户在执行代码之前将代码保存在一个文件中，请将# filename: <filename>放在代码块中作为第一行。不要在一个响应中包含多个代码块。不要要求用户复制和粘贴结果。相反，在相关的时候使用‘print’函数输出。查看用户返回的执行结果。
如果结果表明存在错误，则修复错误并再次输出代码。建议使用完整的代码，而不是部分代码或代码更改。如果错误无法修复，或者即使成功执行了代码，任务也没有解决，那么就分析问题，重新审视您的假设，收集所需的其他信息，然后考虑另一种尝试方法。
当你找到答案时，仔细核实答案。如果可能的话，在你的回答中包括可证实的证据。
当一切都完成后，最后回复“TERMINATE”。
"""
```

&emsp;&emsp;`UserProxyAgent` 则是一个用户代理，它的作用是充当用户的代表，模拟用户的行为。默认情况下在每次交互时请求人类输入作为代理的回复，并且还具有执行代码和调用函数或工具的能力。默认情况下禁用基于大模型的响应。但是也可以通过将`llm_config`设置为与推理配置对应的字典来启用它。当`llm_config`设置配置具体的实例时， `UserProxyAgent`可以在不执行代码时使用LLM生成回复。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501081700069.png" width=100%></div>

```python
class UserProxyAgent(ConversableAgent):

    # Default UserProxyAgent.description values, based on human_input_mode
    DEFAULT_USER_PROXY_AGENT_DESCRIPTIONS = {
        "ALWAYS": "An attentive HUMAN user who can answer questions about the task, and can perform tasks such as running Python code or inputting command line commands at a Linux terminal and reporting back the execution results.",
        "TERMINATE": "A user that can run Python code or input command line commands at a Linux terminal and report back the execution results.",
        "NEVER": "A computer terminal that performs no other action than running Python scripts (provided to it quoted in ```python code blocks), or sh shell scripts (provided to it quoted in ```sh code blocks).",
    }

    def __init__(
        self,
        name: str,
        is_termination_msg: Optional[Callable[[Dict], bool]] = None,
        max_consecutive_auto_reply: Optional[int] = None,
        human_input_mode: Literal["ALWAYS", "TERMINATE", "NEVER"] = "ALWAYS",
        function_map: Optional[Dict[str, Callable]] = None,
        code_execution_config: Union[Dict, Literal[False]] = {},
        default_auto_reply: Optional[Union[str, Dict, None]] = "",
        llm_config: Optional[Union[Dict, Literal[False]]] = False,
        system_message: Optional[Union[str, List]] = "",
        description: Optional[str] = None,
    ):
```

&emsp;&emsp;对应的中文版本：

```python
DEFAULT_USER_PROXY_AGENT_DESCRIPTIONS = {
    "ALWAYS": "一个全神贯注的人工用户，可以回答有关任务的问题，并能执行任务，例如运行 Python 代码或在 Linux 终端输入命令并报告执行结果。",
    "TERMINATE": "一个用户，可以运行 Python 代码或在 Linux 终端输入命令并报告执行结果。",
    "NEVER": "一个计算机终端，除了运行 Python 脚本（以 ```python 代码块 形式提供）或 sh shell 脚本（以 ```sh 代码块 形式提供）外，不执行任何其他操作。",
}
```

&emsp;&emsp;两个`ConversableAgent`的子类定义形式如下图所示：

> 图片来源于 AutoGen 官方文档：https://microsoft.github.io/autogen/0.2/docs/tutorial/conversation-patterns

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501071522099.png" width=100%></div>

In [12]:
import autogen

# 过滤出包含 'ollama' 标签的模型配置
filter_model = {"tags": ["deepseek"]}

config_model = autogen.filter_config(
    config_list=llm_config["config_list"], 
    filter_dict=filter_model)

In [13]:
import os
from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import DockerCommandLineCodeExecutor

# 用大模型配置创建一个名为“assistant”的AssistantAgent实例。
assistant = AssistantAgent(name="assistant",
                           llm_config={"config_list": config_model}
                          )

In [14]:
import os
from autogen.coding import DockerCommandLineCodeExecutor

# 定义用于存储代码和结果的文件夹名称
folder_name = "code_executor_output_docker_dir"

# 获取当前工作目录
current_dir = os.getcwd()

# 将当前工作目录与文件夹名称结合，得到完整的文件夹路径
docker_folder_path = os.path.join(current_dir, folder_name)

# 如果文件夹不存在，则创建该文件夹
os.makedirs(docker_folder_path, exist_ok=True)


# 创建Docker命令行代码执行器
code_executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # 使用给定的docker映像名执行代码。
    timeout=10,
    work_dir=docker_folder_path,  
)

In [16]:
user_proxy = UserProxyAgent(name="user_proxy", 
                            code_execution_config={"executor": code_executor}
                           )

&emsp;&emsp;正确构建两个代理后，就可以通过初始化方法`initiate_chat`启动多代理会话会话，如下代码所示：

In [17]:
# 助手接收来自用户的消息，其中包含任务描述
user_proxy.initiate_chat(
    assistant,
    # 今天是几号？哪只大型科技股今年迄今涨幅最大？收益是多少
    message="""What date is today? Which big tech stock has the largest year-to-date gain this year? How much is the gain?？""", 
)

user_proxy (to assistant):

What date is today? Which big tech stock has the largest year-to-date gain this year? How much is the gain?？

--------------------------------------------------------------------------------
assistant (to user_proxy):

To solve this task, we will break it down into two parts:

1. **Get today's date**: This can be done using Python's `datetime` module.
2. **Find the big tech stock with the largest year-to-date (YTD) gain**: This requires fetching stock data. We can use the `yfinance` library to get stock data for major tech companies.

Let's start with the first part:

### Step 1: Get today's date
```python
# filename: get_today_date.py
from datetime import datetime

# Get today's date
today = datetime.today().strftime('%Y-%m-%d')
print(f"Today's date is: {today}")
```

Please execute the above code to get today's date. Once you have the date, we can proceed to the next step.

TERMINATE

------------------------------------------------------------------------

Replying as user_proxy. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: Today's date is: 2025-01-08


--------------------------------------------------------------------------------
assistant (to user_proxy):

Great! Today's date is **2025-01-08**. Now, let's proceed to the second part of the task: finding the big tech stock with the largest year-to-date (YTD) gain and calculating the gain.

### Step 2: Fetch stock data and calculate YTD gains
We will use the `yfinance` library to fetch stock data for major tech companies (e.g., Apple, Microsoft, Amazon, Google, Tesla, etc.). Then, we will calculate the YTD gain for each stock and determine the one with the largest gain.

Here is the Python code to achieve this:

```python
# filename: get_ytd_gain.py
import yfinance as yf
from datetime import datetime

# Define the list of big tech stocks (tickers)
tech_s

Replying as user_proxy. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': 'What date is today? Which big tech stock has the largest year-to-date gain this year? How much is the gain?？', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'To solve this task, we will break it down into two parts:\n\n1. **Get today\'s date**: This can be done using Python\'s `datetime` module.\n2. **Find the big tech stock with the largest year-to-date (YTD) gain**: This requires fetching stock data. We can use the `yfinance` library to get stock data for major tech companies.\n\nLet\'s start with the first part:\n\n### Step 1: Get today\'s date\n```python\n# filename: get_today_date.py\nfrom datetime import datetime\n\n# Get today\'s date\ntoday = datetime.today().strftime(\'%Y-%m-%d\')\nprint(f"Today\'s date is: {today}")\n```\n\nPlease execute the above code to get today\'s date. Once you have the date, we can proceed to the next step.\n\nTERMINATE', 'role': 'user', 'name': 'assistant'}, {'content': "exitcode: 0 (execut

1. 助手从 user_proxy 接收一条消息，其中包含任务描述。
2. 然后，助手尝试编写 Python 代码来解决任务，并将响应发送到 user_proxy。
3. 一旦 user_proxy 收到助手的响应，它会尝试通过请求人工输入或准备自动生成的回复来进行回复。如果未提供人工输入，则 user_proxy 会执行代码并将结果用作自动回复。
4. 然后，助手为 user_proxy 生成进一步的响应。然后 user_proxy 可以决定是否终止会话。如果不是，则重复步骤 3 和 4

初始化步骤完成后，对话可以自动进行。下面是 user_proxy 和助手如何自主协作解决上述任务的直观说明：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202501071432608.png" width=100%></div>

&emsp;&emsp;总的来看，`AssistantAgent` 是主动的，它根据任务要求自动生成响应；而 `UserProxyAgent` 是被动的，它只是模拟用户的行为并转发请求或反馈。所以两个子类各自的应用场景区别也比较大，`AssistantAgent` 通常用于提供帮助、执行任务和解决问题；而 `UserProxyAgent` 则是用于代表用户发起问题、请求和对话。

In [21]:
# 创建助手代理（AssistantAgent）
marketing_assistant = AssistantAgent(
    name="MarketingAssistant",
    system_message="你是一名市场营销专家，将提供营销策略方面的见解。",
    llm_config={"config_list": config_model}
)

# 创建用户代理（UserProxyAgent）
user_proxy = UserProxyAgent(
    name="UserProxy",
    system_message="你是一位客户，询问有关营销策略的问题。",
)

# 启动对话：用户代理提问，助手代理提供答案
result = user_proxy.initiate_chat(
    recipient=marketing_assistant,
    message="在数字领域提高品牌知名度的最佳策略是什么？？",
    summary_method="reflection_with_llm",  # 使用 LLM 进行总结
    summary_args={
        "summary_role": "system",
        "summary_prompt": "请回复一份有效的数字营销品牌意识策略的摘要, 简短而明确。"
    },
)

# 输出对话结果
print(result)

UserProxy (to MarketingAssistant):

在数字领域提高品牌知名度的最佳策略是什么？？

--------------------------------------------------------------------------------
MarketingAssistant (to UserProxy):

在数字领域提高品牌知名度需要采取多管齐下的策略，结合各种在线营销渠道和技术。以下是一些最有效的策略：

**1. 搜索引擎优化 (SEO)：**

* **关键词研究：** 确定与您的品牌和产品相关的目标关键词。
* **网站优化：** 优化您的网站内容、元数据和网站结构，以提高搜索引擎排名。
* **内容营销：** 创建高质量、信息丰富且与目标受众相关的内容，以吸引自然流量。
* **链接建设：** 从信誉良好的网站获取高质量的反向链接，以提高您的网站权威性。

**2. 社交媒体营销：**

* **选择合适的平台：** 确定您的目标受众最活跃的社交媒体平台。
* **创建引人入胜的内容：** 分享与您的品牌相关且对您的受众有价值的内容，例如博客文章、信息图表、视频和图像。
* **与您的受众互动：** 回复评论和消息，参与对话，并举办竞赛和赠品。
* **利用社交媒体广告：** 使用有针对性的广告来覆盖更广泛的受众并提高品牌知名度。

**3. 内容营销：**

* **创建有价值的内容：** 开发与您的目标受众相关且信息丰富的内容，例如博客文章、电子书、白皮书和案例研究。
* **推广您的内容：** 通过社交媒体、电子邮件营销和其他渠道分享您的内容，以覆盖更广泛的受众。
* **与影响者合作：** 与您所在行业的影响者合作，以推广您的内容并提高品牌知名度。

**4. 付费广告：**

* **搜索引擎营销 (SEM)：** 在搜索引擎结果页面上投放付费广告，以覆盖正在搜索与您的品牌相关的关键词的用户。
* **社交媒体广告：** 在社交媒体平台上投放有针对性的广告，以覆盖特定的受众群体。
* **展示广告：** 在相关网站和应用程序上投放横幅广告和其他展示广告，以提高品牌知名度。

**5. 电子邮件营销：**

* **建立您的电子邮件列表：** 通过您的网站、社交媒体渠道和其他接触点收集电子邮件地址。
*

Replying as UserProxy. Provide feedback to MarketingAssistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': '在数字领域提高品牌知名度的最佳策略是什么？？', 'role': 'assistant', 'name': 'UserProxy'}, {'content': '在数字领域提高品牌知名度需要采取多管齐下的策略，结合各种在线营销渠道和技术。以下是一些最有效的策略：\n\n**1. 搜索引擎优化 (SEO)：**\n\n* **关键词研究：** 确定与您的品牌和产品相关的目标关键词。\n* **网站优化：** 优化您的网站内容、元数据和网站结构，以提高搜索引擎排名。\n* **内容营销：** 创建高质量、信息丰富且与目标受众相关的内容，以吸引自然流量。\n* **链接建设：** 从信誉良好的网站获取高质量的反向链接，以提高您的网站权威性。\n\n**2. 社交媒体营销：**\n\n* **选择合适的平台：** 确定您的目标受众最活跃的社交媒体平台。\n* **创建引人入胜的内容：** 分享与您的品牌相关且对您的受众有价值的内容，例如博客文章、信息图表、视频和图像。\n* **与您的受众互动：** 回复评论和消息，参与对话，并举办竞赛和赠品。\n* **利用社交媒体广告：** 使用有针对性的广告来覆盖更广泛的受众并提高品牌知名度。\n\n**3. 内容营销：**\n\n* **创建有价值的内容：** 开发与您的目标受众相关且信息丰富的内容，例如博客文章、电子书、白皮书和案例研究。\n* **推广您的内容：** 通过社交媒体、电子邮件营销和其他渠道分享您的内容，以覆盖更广泛的受众。\n* **与影响者合作：** 与您所在行业的影响者合作，以推广您的内容并提高品牌知名度。\n\n**4. 付费广告：**\n\n* **搜索引擎营销 (SEM)：** 在搜索引擎结果页面上投放付费广告，以覆盖正在搜索与您的品牌相关的关键词的用户。\n* **社交媒体广告：** 在社交媒体平台上投放有针对性的广告，以覆盖特定的受众群体。\n* **展示广告：** 在相关网站和应用程序上投放横幅广告和其他展示广告，以提高品牌知名度。\n\n**5. 电子邮件营销：**\n\n* **建立您的电子邮件列表：** 通过您的网站、社交媒体渠道和其他接触点收集电子邮件地址。\n* **发送有针对性的电子邮

In [22]:
print(result.summary)

在数字领域提高品牌知名度的最佳策略包括：

1. **内容营销**：创建高质量、有价值的内容（如博客、视频、信息图表），吸引目标受众并提升品牌曝光。
2. **社交媒体营销**：在目标受众活跃的平台上（如Instagram、LinkedIn、TikTok）定期发布内容，与用户互动，扩大品牌影响力。
3. **SEO优化**：通过关键词优化、高质量内容和反向链接，提升品牌在搜索引擎中的排名。
4. **付费广告**：利用Google Ads、社交媒体广告等精准投放工具，快速提升品牌曝光。
5. **网红合作**：与行业相关的网红或意见领袖合作，借助他们的影响力扩大品牌知名度。
6. **电子邮件营销**：通过个性化邮件与潜在客户保持联系，传递品牌价值。
7. **用户生成内容（UGC）**：鼓励用户分享与品牌相关的内容，增强品牌可信度和参与度。
8. **数据分析与优化**：持续跟踪营销效果，优化策略以提高ROI。

结合这些策略，品牌可以更有效地在数字领域提升知名度和影响力。
